# read me

1. Executing this file requires Python 3.8.5.

2. The virtual environment must be activated.

3. You must create result storage folders such as S0, S1, etc. in the root directory; otherwise, the results cannot be saved.

4. Running this file requires a Gurobi commercial (professional) license; the free license is insufficient to support large-scale computations.

In [39]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from openpyxl import load_workbook
import gurobipy as gp
import copy
from scipy.stats import uniform, norm, expon, lognorm, beta
import os

# Indices

In [2]:
# timeframe from 2020 to 2060
years = pd.read_excel('indices.xlsx', sheet_name='years', header=None, index_col=None)[0].to_list()
# name (and order) of technologies
technologies = pd.read_excel('indices.xlsx', sheet_name='technologies', header=None, index_col=None)[0].to_list()
# process name
processes = pd.read_excel('indices.xlsx', sheet_name='processes', header=None, index_col=None)[0].to_list()
METH_processes=pd.read_excel('indices.xlsx', sheet_name='METH_processes', header=None, index_col=None)[0].to_list()
HVC_processes=pd.read_excel('indices.xlsx', sheet_name='HVC_processes', header=None, index_col=None)[0].to_list()

#province
province = pd.read_excel('indices.xlsx', sheet_name='province', header=None, index_col=None)[0].to_list()

#products
chemicals=['AMM','METH','Oil']
HVCs=['ETH','PRO','BTX']
#scenario
scenario=['S0','S1','S2','S3','S4','S5']
#cost
costs=['CAP','OM','FS','CCS']
#the relationship between technologies and processes
TP_relation_arr=np.array(pd.read_excel('indices.xlsx',sheet_name='t-p-relation',header=None,index_col=None).iloc[1:,1:])
TP_relation={}
for i in range(len(TP_relation_arr)):
    TP_relation[technologies[i]]={}
    process_list=[]
    for j in range(len(TP_relation_arr[i])):
        if TP_relation_arr[i][j]==1:
            process_list.append(processes[j])
    TP_relation[technologies[i]]=process_list
PT_relation={}
for i in range(len(np.transpose(TP_relation_arr))):
    PT_relation[processes[i]]={}
    tech_list=[]
    for j in range(len(TP_relation_arr)):
        if TP_relation_arr[j][i]==1:
            tech_list.append(technologies[j])
        PT_relation[processes[i]]=tech_list
PP_relation_arr=np.array(pd.read_excel('indices.xlsx',sheet_name='p-p-relation',header=None,index_col=None).iloc[1:,1:])
PP_relation={}
for i in range(len(METH_processes)):
    PP_relation[METH_processes[i]]={}
    process_list=[]
    for j in range(len(processes)):
        if PP_relation_arr[i][j]==1:
            process_list.append(processes[j])
    PP_relation[METH_processes[i]]=process_list
process_can_es=6
es_index={}
for i in range(process_can_es):
    es_index[processes[process_can_es+i]]=processes[i]
CCS_index={
    'BG':'BG-CCS',
    'CG':'CG-CCS',
    'SMR':'SMR-CCS'
}
CCS_re_index={
    'BG-CCS':'BG',
    'CG-CCS':'CG',
    'SMR-CCS':'SMR'
}
FS_index={
    'Grid-PEM':'Grid-AE',
    'BG-CCS':'BG',
    'CG-CCS':'CG',
    'SMR-CCS':'SMR'
}
H2_groups={'Wind': ['Wind-es-AE', 'Wind-es-PEM', 'Wind-wes-AE', 'Wind-wes-PEM'],
  'Solar': ['Solar-es-AE', 'Solar-es-PEM', 'Solar-wes-AE', 'Solar-wes-PEM'],
  'Hydro': ['Hydro-es-AE', 'Hydro-es-PEM', 'Hydro-wes-AE', 'Hydro-wes-PEM'],
  'Nuclear': ['Nuclear-wes-AE',
   'Nuclear-wes-PEM',
   'Nuclear-wes-SOEC',
   'Nuclear-THP'],
  'Grid':['Grid-AE','Grid-PEM'],
  'BG': ['BG'],
  'CG': ['CG'],
  'SMR': ['SMR'],
  'EXT': ['EXT'],
  'BG-CCS':['BG-CCS'],
  'CG-CCS':['CG-CCS'],
  'SMR-CCS':['SMR-CCS'],
  
  'AE': ['Wind-es-AE',
   'Solar-es-AE',
   'Hydro-es-AE',
   'Wind-wes-AE',
   'Solar-wes-AE',
   'Hydro-wes-AE',
   'Nuclear-wes-AE',
   'Grid-AE'],
  'PEM': ['Wind-es-PEM',
   'Solar-es-PEM',
   'Hydro-es-PEM',
   'Wind-wes-PEM',
   'Solar-wes-PEM',
   'Hydro-wes-PEM',
   'Nuclear-wes-PEM',
   'Grid-PEM'],
  'SOEC': ['Nuclear-wes-SOEC'],
           
  'Energy_stroage': ['Wind-es-AE',
   'Wind-es-PEM',
   'Solar-es-AE',
   'Solar-es-PEM',
   'Hydro-es-AE',
   'Hydro-es-PEM',],
  'Non_Energy_stroage':['Wind-wes-AE',
   'Wind-wes-PEM',
   'Solar-wes-AE',
   'Solar-wes-PEM',
   'Hydro-wes-AE',
   'Hydro-wes-PEM',
   'Nuclear-wes-AE',
   'Nuclear-wes-PEM',
   'Nuclear-wes-SOEC']
}
HVC_relations={
    'ETH':['NSC','MTO','ESC'],
    'PRO':['NSC','MTO','CC','PDH'],
    'BTX':['NSC','CR']
}
def get_distance():
    df=pd.read_excel('indices.xlsx',sheet_name='adjacency')
    adjacency_dict = {}
    for province in df.index:
        neighbors = df.loc[province][df.loc[province] == 1].index.tolist()
        adjacency_dict[df.loc[province]['Province']] = neighbors
    for province in adjacency_dict:
        adjacency_dict[province] = [neighbor for neighbor in adjacency_dict[province] if neighbor != province]
    df_distance=pd.read_excel('indices.xlsx',sheet_name='distance')
    return adjacency_dict,df_distance
def get_interpolated(values):
    if len(values)==5:
        num_points = 10
    else:
        num_points=5
    interpolated_values = []
    interpolated_values.append(values[0])
    for i in range(len(values) -1):
        start_value = values[i]
        end_value = values[i + 1]
        add_value=(end_value-start_value)/(num_points)
        for j in range(num_points):
            interpolated_values.append(start_value+(j+1)*add_value)
    return interpolated_values


In [3]:
adjacency_dict,df_distance=get_distance()

# Parameters

### COST

In [4]:
def get_cost_data(ele_cost_level='M',bio_cost='M'):
    cost=pd.read_excel('data.xlsx',sheet_name='tech-cost',index_col=None)
    cost_data=cost.iloc[:, 3:]
    if ele_cost_level=='H':
        cost_change = [1, 1.25, 1.2, 1.2, 1.2]
    elif ele_cost_level=='L':
        cost_change = [1, 0.75, 0.8, 0.8, 0.8]
    elif ele_cost_level=='h':
        cost_change = [1, 1.125, 1.1, 1.1, 1.1]
    elif ele_cost_level=='l':
        cost_change = [1, 0.875, 0.9, 0.9, 0.9]
    else:
        cost_change=[1,1,1,1,1]
    
    if bio_cost=='h':
        bio_cost_change=[1, 1.125, 1.1, 1.1, 1.1]
    elif bio_cost=='l':
        bio_cost_change=[1, 0.875, 0.9, 0.9, 0.9]
    else:
        bio_cost_change=[1,1,1,1,1]
    cc = get_interpolated(cost_change)
    bio_cc=get_interpolated(bio_cost_change)
    int_years = [2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]
    new_columns = [str(int_year) for int_year in range(2020, 2061)]
    interpolated_cost_data = pd.DataFrame(columns=new_columns)
    for idx, row in cost_data.iterrows():
        data = row.to_numpy()
        interpolated_values = get_interpolated(list(data))
        interpolated_cost_data = interpolated_cost_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)

    CAP_cost=interpolated_cost_data[0:13]
    OM_cost=interpolated_cost_data[13:26]
    CAP = {}
    OM={}
    for t in years:
        CAP[t] = {}
        OM[t]={}
        k=0
        for j in technologies:
            value1 = CAP_cost.at[k,str(t)]
            value2=OM_cost.at[13+k,str(t)]
            k=k+1
            CAP[t][j] = value1
            OM[t][j]=value2
            if j in ['AE','PEM','SOEC']:
                CAP[t][j] = CAP[t][j]*cc[t-2020]
                OM[t][j]=OM[t][j]*cc[t-2020]

    FS=pd.read_excel('data.xlsx',sheet_name='FS-cost',index_col=None)
    FS_data=FS.iloc[:, 2:]
    int_years = [2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]
    new_columns = [str(int_year) for int_year in range(2020, 2061)]
    interpolated_FS_data = pd.DataFrame(columns=new_columns)

    for idx, row in FS_data.iterrows():
        data = row.to_numpy()
        interpolated_values = get_interpolated(list(data))
        interpolated_FS_data = interpolated_FS_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)
    FS = {}
    for t in years:
        FS[t] = {}
        k=0
        for i in processes:
            value1 = interpolated_FS_data.at[k,str(t)]
            k=k+1
            FS[t][i] = value1
            if i in ['BG','BG-CCS']:
                FS[t][i] =FS[t][i] *bio_cc[t-2020]
    return CAP,OM,FS

def get_next_cost_data(level):
    if level=='h':
        level='H'
        a=0.5
    elif level=='l':
        level='L'
        a=0.5
    else:
        a=1
    NC=pd.read_excel('data.xlsx',sheet_name='next-cost-{}'.format(level),index_col=None)
    NC_basic=pd.read_excel('data.xlsx',sheet_name='next-cost-M',index_col=None)
    NC_data=NC.iloc[:, 3:]
    NC_basic_data=NC_basic.iloc[:, 3:]
    int_years = [2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]
    new_columns = [str(int_year) for int_year in range(2020, 2061)]
    interpolated_NC_data = pd.DataFrame(columns=new_columns)
    interpolated_NC_basic_data = pd.DataFrame(columns=new_columns)

    for idx, row in NC_data.iterrows():
        data = row.to_numpy()
        interpolated_values = get_interpolated(list(data))
        interpolated_NC_data = interpolated_NC_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)
    for idx, row in NC_basic_data.iterrows():
        data = row.to_numpy()
        interpolated_values = get_interpolated(list(data))
        interpolated_NC_basic_data = interpolated_NC_basic_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)
    
    
    

    NC = {}
    for t in years:
        NC[t] = {}
        k=0
        for i in ['AMM','METH1','METH2','METH3']:#3 METH processes:CO-based,industrial CO2-based and DAC CO2-based
            NC[t][i]={}
            for j in ['CAP','OM','FS']:
                value1 = interpolated_NC_data.at[k,str(t)]*a+interpolated_NC_basic_data.at[k,str(t)]*(1-a)
                k=k+1
                NC[t][i][j] = value1
    return NC

def interpolate_fs_cost(years, values):
    full_years = np.arange(min(years), max(years) + 1)
    interpolated_values = np.interp(full_years, years, values)
    return {year: value for year, value in zip(full_years, interpolated_values)}

def get_hvc_parameters():
    HVC_para = pd.read_excel('data.xlsx', sheet_name='HVC_para', index_col=None)[['process', 'var', 'unit', 'value']]
    
    HVC_para_dict = {}
    
    for hvc in ['ETH', 'PRO', 'BTX']:
        HVC_para_dict[hvc] = {}
        HVC_para_dict[hvc]['Embodied CF']=HVC_para.loc[ (HVC_para['process'] == 'Embodied CF')&(HVC_para['var'] == hvc), 'value'].tolist()[0]
        for hvc_process in HVC_relations[hvc]:
            HPC = hvc + '-' + hvc_process
            HVC_para_dict[hvc][HPC] = {}
            for para in ['CAP', 'OM', 'CF']:
                huilv = 7 if para in ['CAP', 'OM'] else 1
                HVC_para_dict[hvc][HPC][para] = huilv * HVC_para.loc[
                    ((HVC_para['process'] == hvc_process) & (HVC_para['var'] == para)), 'value'].tolist()[0]
            years = [2020, 2030, 2040, 2050, 2060]
            HVC_para_dict[hvc][HPC]['FS-cost'] = {}
            values = [7 * HVC_para.loc[
                ((HVC_para['process'] == hvc_process) & (HVC_para['var'] == f'FS_cost-{y}')), 'value'].tolist()[0] 
                for y in years]
            HVC_para_dict[hvc][HPC]['FS-cost'] = interpolate_fs_cost(years, values)

            HVC_para_dict[hvc][HPC]['out/in'] = {}
            values = [HVC_para.loc[
                ((HVC_para['process'] == hvc_process) & (HVC_para['var'] == f'out/in-{y}')), 'value'].tolist()[0] 
                for y in years]
            HVC_para_dict[hvc][HPC]['out/in'] = interpolate_fs_cost(years, values)
    
    HVC_para_dict['METH-Embodied CF']=1.375
    values=np.array(pd.read_excel('data.xlsx', sheet_name='HVC_CCS_ratio', index_col=None)[[2020,2030,2040,2050,2060]])[0].tolist()
    HVC_para_dict['CCS_end_ratio']=interpolate_fs_cost([2020,2030,2040,2050,2060], values)
    return HVC_para_dict

def get_trans_cost():
    df=pd.read_excel('data.xlsx','Trans_cost')
    trans_cost = {}
    for chem in ['AMM', 'METH', 'H2']:
        trans_cost[chem] = {}
        for _, row in df.iterrows():
            trans_cost[chem][row['DIS']]=row[chem]
            
    return trans_cost


### lifetime

In [5]:
def get_LT_data():
    lifetime=pd.read_excel('data.xlsx',sheet_name='lifetime',index_col=None)
    lifetime_data=lifetime.iloc[:len(technologies)+len(HVC_processes), 2:]
    int_years = [2020,  2030,  2040,  2050,  2060]
    new_columns = [str(int_year) for int_year in range(2020, 2061)]
    interpolated_lifetime_data = pd.DataFrame(columns=new_columns)

    for idx, row in lifetime_data.iterrows():
        data = row.to_numpy()
        interpolated_values =get_interpolated(list(data))
        interpolated_lifetime_data = interpolated_lifetime_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)


    LT={}
    for t in years:
        LT[t] = {}
        k=0
        for j in technologies+HVC_processes:
            value = interpolated_lifetime_data.at[k,str(t)]
            k=k+1
            LT[t][j] = value
    return LT

### OH max

In [6]:
def get_OH_data():
    OH_max=pd.read_excel('data.xlsx',sheet_name='OH_other',index_col=None)
    OH_WSH=pd.read_excel('data.xlsx',sheet_name='OH_WSH',index_col=None)
    OH_max_data=OH_max.iloc[:len(technologies), 2:]
    int_years = [2020,  2030,  2040,  2050,  2060]
    new_columns = [str(int_year) for int_year in range(2020, 2061)]
    interpolated_OH_max_data = pd.DataFrame(columns=new_columns)

    for idx, row in OH_max_data.iterrows():
        data = row.to_numpy()
        interpolated_values =get_interpolated(list(data))
        interpolated_OH_max_data = interpolated_OH_max_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)


    OH_max={}
    for t in years:
        OH_max[t] = {}
        k=0
        for j in technologies:
            if j not in ['Wind','Solar','Hydro']:
                OH_max[t][j]={}
                for p in province:
                    value = interpolated_OH_max_data.at[k,str(t)]
                    OH_max[t][j][p] = value
            if j in ['Wind','Solar','Hydro']:
                OH_max[t][j]={}
                for p in province:
                    OH_max[t][j][p] = OH_WSH.loc[OH_WSH['PROVINCE'] == p, '{}-OH'.format(j)].tolist()[0]
                
            k=k+1
        
            

    return OH_max

### HP-factor

In [7]:
def get_HP_data():
    HP=pd.read_excel('data.xlsx',sheet_name='HP',index_col=None)
    HP_data=HP.iloc[:, 2:]
    int_years = [2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]
    new_columns = [str(int_year) for int_year in range(2020, 2061)]
    interpolated_HP_data = pd.DataFrame(columns=new_columns)

    for idx, row in HP_data.iterrows():
        data = row.to_numpy()
        interpolated_values = get_interpolated(list(data))
        interpolated_HP_data = interpolated_HP_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)
    HP = {}
    for t in years:
        HP[t] = {}
        k=0
        for i in processes:
            value1 = interpolated_HP_data.at[k,str(t)]
            k=k+1
            HP[t][i] = value1
    return HP

### CF-factor

In [8]:
def get_CF_data():
    CF=pd.read_excel('data.xlsx',sheet_name='CF',index_col=None)
    CF_data=CF.iloc[:, 2:]
    int_years = [2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]
    new_columns = [str(int_year) for int_year in range(2020, 2061)]
    interpolated_CF_data = pd.DataFrame(columns=new_columns)

    for idx, row in CF_data.iterrows():
        data = row.to_numpy()
        interpolated_values = get_interpolated(list(data))
        interpolated_CF_data = interpolated_CF_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)
    CF= {}
    for t in years:
        CF[t] = {}
        k=0
        for i in processes:
            value1 = interpolated_CF_data.at[k,str(t)]
            k=k+1
            CF[t][i] = value1
    CCS_delta_CF={}
    for t in years:
        CCS_delta_CF[t]={}
        for i in processes:
            if i=='BG':
                v10=CF[t][i]
            if i=='CG':
                v20=CF[t][i]
            if i=='SMR':
                v30=CF[t][i]
            if i=='BG-CCS':
                v11=CF[t][i]
            if i=='CG-CCS':
                v21=CF[t][i]
            if i=='SMR-CCS':
                v31=CF[t][i]
        CCS_delta_CF[t]['BG-CCS']=-v11+v10
        CCS_delta_CF[t]['CG-CCS']=v20-v21
        CCS_delta_CF[t]['SMR-CCS']=v30-v31
    return CF,CCS_delta_CF

In [9]:
def get_CF_METH_data():
    CF=pd.read_excel('data.xlsx',sheet_name='CF_METH',index_col=None)
    CF_data=CF.iloc[:, 2:]
    int_years = [2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]
    new_columns = [str(int_year) for int_year in range(2020, 2061)]
    interpolated_CF_data = pd.DataFrame(columns=new_columns)

    for idx, row in CF_data.iterrows():
        data = row.to_numpy()
        interpolated_values = get_interpolated(list(data))
        interpolated_CF_data = interpolated_CF_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)
    CF= {}
    for t in years:
        CF[t] = {}
        k=0
        for i in METH_processes:
            value1 = interpolated_CF_data.at[k,str(t)]
            k=k+1
            CF[t][i] = value1
    CCS_delta_CF={}
    for t in years:
        CCS_delta_CF[t]={}
        for i in METH_processes:
            if i=='Biomass-based':
                v10=CF[t][i]
            if i=='Coal-based':
                v20=CF[t][i]
            if i=='Gas-based':
                v30=CF[t][i]
            if i=='Biomass-based-CCS':
                v11=CF[t][i]
            if i=='Coal-based-CCS':
                v21=CF[t][i]
            if i=='Gas-based-CCS':
                v31=CF[t][i]
        CCS_delta_CF[t]['Biomass-based-CCS']=-v11+v10
        CCS_delta_CF[t]['Coal-based-CCS']=v20-v21
        CCS_delta_CF[t]['Gas-based-CCS']=v30-v31
    return CF,CCS_delta_CF

### EXCAP

In [10]:
def get_EXCAP_data():
    EXCAP=pd.read_excel('data.xlsx',sheet_name='EXCAP',index_col=None)
    EXCAP_data=EXCAP.iloc[:, 2:]
    int_years = [2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]
    new_columns = [str(int_year) for int_year in range(2020, 2061)]
    interpolated_EXCAP_data = pd.DataFrame(columns=new_columns)

    for idx, row in EXCAP_data.iterrows():
        data = row.to_numpy()
        interpolated_values = get_interpolated(list(data))
        interpolated_EXCAP_data = interpolated_EXCAP_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)
    EXCAP= {}
    for t in years:
        EXCAP[t] = {}
        k=0
        for i in technologies+HVC_processes:
            value1 = interpolated_EXCAP_data.at[k,str(t)]
            k=k+1
            EXCAP[t][i] = value1
    EXCAP_process={}
    for t in years:
        EXCAP_process[t]={}
        for i in processes:
            if i=='CG' or i=='SMR' or i=='EXT':
                EXCAP_process[t][i]=EXCAP[t][i]
            else:
                EXCAP_process[t][i]=0
    return EXCAP,EXCAP_process

### POT

In [11]:
def get_POT_data():
    EXP_chemical_ratio,chemical_EXP_ratio=get_two_ratio_2020()
    SMR_ratio={}
    EXT_ratio={}
    SMR_ratio['tot']=0
    EXT_ratio['tot']=0
    for p in province:
        SMR_ratio[p]=sum(EXP_chemical_ratio['SMR'][c][p] for c in chemicals)
        SMR_ratio['tot']=SMR_ratio['tot']+SMR_ratio[p]
        EXT_ratio[p]=sum(EXP_chemical_ratio['EXT'][c][p] for c in chemicals)
        EXT_ratio['tot']=EXT_ratio['tot']+EXT_ratio[p]
    Hydro_POT=pd.read_excel('data.xlsx',sheet_name='Hydro-POT',index_col=None)
    Hydro_ratio={}
    Nuclear_ratio={}
    Hydro_ratio['tot']=0
    Nuclear_ratio['tot']=100
    for p in province:
        Hydro_ratio[p]=Hydro_POT.loc[Hydro_POT['PROVINCE'] == p, 'Hydro_2023'].tolist()[0]
        Nuclear_ratio[p]=(Hydro_POT.loc[Hydro_POT['PROVINCE'] == p, 'Nuclear_2023'].tolist()[0])/Hydro_POT['Nuclear_2023'].sum()
        Hydro_ratio['tot']=Hydro_ratio['tot']+Hydro_ratio[p]
    
    POT=pd.read_excel('data.xlsx',sheet_name='POT',index_col=None)
    POT_data=POT.iloc[:, 2:]
    int_years = [2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]
    new_columns = [str(int_year) for int_year in range(2020, 2061)]
    interpolated_POT_data = pd.DataFrame(columns=new_columns)

    for idx, row in POT_data.iterrows():
        data = row.to_numpy()
        interpolated_values = get_interpolated(list(data))
        interpolated_POT_data = interpolated_POT_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)
    POT= {}
    for t in years:
        POT[t] = {}
        k=0
        for i in technologies:
            POT[t][i]={}
            value1 = interpolated_POT_data.at[k,str(t)]
            for p in province:
                if i=='SMR':
                    value2=value1*SMR_ratio[p]/SMR_ratio['tot']
                    POT[t][i][p] = value2
                elif i=='EXT':
                    value2=value1*EXT_ratio[p]/EXT_ratio['tot']
                    POT[t][i][p] = value2
                elif i=='Hydro':
                    value2=value1*Hydro_ratio[p]/Hydro_ratio['tot']
                    POT[t][i][p] = value2
                elif i=='Nuclear':
                    value2=value1*Nuclear_ratio[p]
                    POT[t][i][p] = value2
                else:
                    POT[t][i][p]=value1
            k=k+1
            
    return POT

In [12]:
def get_Wind_Solar_POT():
    data=pd.read_excel('data.xlsx',sheet_name='W&S_POT',index_col=None)
    WS_POT={}
    for i in ['Wind','Solar']:
        WS_POT[i]={}
        for p in province:
            WS_POT[i][p]=data.loc[data['PROVINCE']==p,'{}'.format(i)].tolist()[0]
    
    return WS_POT
    

In [13]:
def get_Bio_POT():
    BIO_data=pd.read_excel('data.xlsx',sheet_name='Bio-POT',index_col=None)
    Bio_data={}
    for sce in ['BAU','MAR II','MAR I','MAR I+II']:
        Bio_data[sce]={}
        for p in province:
            Bio_data[sce][p]=BIO_data.loc[BIO_data['PROVINCE'] == p, '{}'.format(sce)].tolist()[0]
    return Bio_data

In [14]:
def get_CO2_POT():
    #industrial CO2 potential
    CO2_POT=pd.read_excel('data.xlsx',sheet_name='Industry_CO2_POT',index_col=None).iloc[:,2:]
    int_years = [2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]
    new_columns = [str(int_year) for int_year in range(2020, 2061)]
    interpolated_CO2POT_data = pd.DataFrame(columns=new_columns)


    for idx, row in CO2_POT.iterrows():
        data = row.to_numpy()
        interpolated_values = get_interpolated(list(data))
        interpolated_CO2POT_data = interpolated_CO2POT_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)
    CO2_POT= {}
    for t in years:
        CO2_POT[t]={}
        for k in range(3):
            CO2_POT[t][['L','M','H'][k]] = interpolated_CO2POT_data.at[k,str(t)]

    return CO2_POT

# CCS cost

In [15]:
def get_CCS_cost_data():
    CCS_cost=pd.read_excel('data.xlsx',sheet_name='CCS_cost',index_col=None)
    CCS_cost_dict = {}
    
    for p in province:
        values = [CCS_cost.loc[
            (CCS_cost['Province'] == p),t].tolist()[0] 
            for t in [2020, 2030, 2040, 2050, 2060]]
        CCS_cost_dict[p]=interpolate_fs_cost([2020, 2030, 2040, 2050, 2060], values)
    return CCS_cost_dict

# energy intensity and energy data

In [16]:
def get_energy_intensity():
    data=pd.read_excel('data.xlsx',sheet_name='energy_intensity',index_col=None)
    energy_intensity={}
    for i in ['AMM','METH']+HVC_processes:
        energy_intensity[i]={}
        for j in ['H_GJ/t','E_MWh/t']:
            energy_intensity[i][j]=data.loc[data['process'] == i, j].tolist()[0]
    return energy_intensity

In [17]:
def get_ele_data():
    ele_CF=pd.read_excel('data.xlsx',sheet_name='electricity_CF',index_col=None)
    ele_cost=pd.read_excel('data.xlsx',sheet_name='electricity_cost',index_col=None)
    
    ele_dict = {}
    ele_dict['CF']={}
    ele_dict['cost']={}
    for p in province:
        values = [ele_CF.loc[
            (ele_CF['Province'] == p),t].tolist()[0] 
            for t in [2020, 2030, 2040, 2050, 2060]]
        ele_dict['CF'][p]=interpolate_fs_cost([2020, 2030, 2040, 2050, 2060], values)
        
        values = [ele_cost.loc[
            (ele_cost['Province'] == p),t].tolist()[0] 
            for t in [2020, 2030, 2040, 2050, 2060]]
        ele_dict['cost'][p]=interpolate_fs_cost([2020, 2030, 2040, 2050, 2060], values)
    return ele_dict

In [18]:
def get_heat_production_para():
    HP_para = pd.read_excel('data.xlsx', sheet_name='fuel_para', index_col=None)[['fuel', 'var', 'value']]
    
    HP_para_dict = {}
    for f in ['Biomass','NG']:
        HP_para_dict[f]={}
        for para in ['heat value','heat eff','CF','CF-CCS','CAP+OM']:
            HP_para_dict[f][para]=HP_para.loc[ (HP_para['fuel'] == f)&(HP_para['var'] == para), 'value'].tolist()[0]
        HP_para_dict[f]['FS']={}
        years = [2020, 2030, 2040, 2050, 2060]
        values = [HP_para.loc[
                ((HP_para['fuel'] == f) & (HP_para['var'] == f'cost-{y}')), 'value'].tolist()[0] 
                for y in years]
        HP_para_dict[f]['FS']=interpolate_fs_cost(years, values)
    return HP_para_dict

### Scenario :demand and carbon proce and CCS-cost

In [19]:


def create_demand_dict():

    df=pd.read_excel('data.xlsx',sheet_name='Demand_v2',index_col=None)

    year_cols = [col for col in df.columns if isinstance(col, int) and 2020 <= col <= 2060]
    

    full_years = list(range(2020, 2061))
    

    if len(year_cols) < len(full_years):

        year_df = df[year_cols].copy()
        col_map = {year: idx for idx, year in enumerate(year_cols)}
        
        for target_year in full_years:
            if target_year not in year_df.columns:

                prev_years = [y for y in year_cols if y < target_year]
                next_years = [y for y in year_cols if y > target_year]
                
                if prev_years and next_years:
                    prev_col = max(prev_years)
                    next_col = min(next_years)
                    
                    t_diff = next_col - prev_col
                    w1 = (next_col - target_year) / t_diff
                    w2 = (target_year - prev_col) / t_diff
                    
                    year_df[target_year] = year_df[prev_col] * w1 + year_df[next_col] * w2
    

    df = pd.concat([df.drop(columns=year_cols), year_df], axis=1)
    

    demand = {}
    
    for (chem, downstream), group in df.groupby(['chemical', 'downstream']):

        chem_data = {year: group[year].values[0] for year in full_years}
        
        if chem not in demand:
            demand[chem] = {}
        demand[chem][downstream] = chem_data

    for chem, downstreams in demand.items():
        tot_data = {}
        for year in full_years:

            tot_data[year] = sum(
                d_data[year] 
                for d_name, d_data in downstreams.items() 
                if d_name != 'Tot' 
            )
        demand[chem]['Tot'] = tot_data
    
    return demand

In [20]:
def get_province_demand_supply_share():
    df = pd.read_excel('data.xlsx', sheet_name='province_share', index_col=None)
    supply_columns = []
    for col in df.columns:
        if "Supply" in col:
            supply_columns.append(col)
    demand_columns = [col for col in df.columns if col != 'PROVINCE' and col not in supply_columns]
    supply_ratio = {}
    demand_ratio = {}

    for _, row in df.iterrows():
        province = row['PROVINCE']

        supply_ratio[province] = {col: row[col] for col in supply_columns}
        demand_ratio[province] = {col: row[col] for col in demand_columns}
    return {
        "Supply_ratio": supply_ratio,
        "Demand_ratio": demand_ratio
    }

In [21]:
def get_SCE_data():
    province_share=pd.read_excel('data.xlsx',sheet_name='province_share',index_col=None)
    #demand
    demand=pd.read_excel('data.xlsx',sheet_name='Demand',index_col=None)
    demand_data=demand.iloc[:, 3:]
    #carbon target
    CP=pd.read_excel('data.xlsx',sheet_name='Carbon-target',index_col=None)
    CP_data=CP.iloc[:, 2:]
    #CDR-cost
    CDR=pd.read_excel('data.xlsx',sheet_name='CDR-cost',index_col=None)
    CDR_data=CDR.iloc[:, 2:]

    int_years = [2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]
    new_columns = [str(int_year) for int_year in range(2020, 2061)]
    interpolated_demand_data = pd.DataFrame(columns=new_columns)
    interpolated_CP_data = pd.DataFrame(columns=new_columns)
    interpolated_CDR_data = pd.DataFrame(columns=new_columns)

    for idx, row in demand_data.iterrows():
        data = row.to_numpy()
        interpolated_values = get_interpolated(list(data))
        interpolated_demand_data = interpolated_demand_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)

    for idx, row in CP_data.iterrows():
        data = row.to_numpy()
        interpolated_values = get_interpolated(list(data))
        interpolated_CP_data = interpolated_CP_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)

    for idx, row in CDR_data.iterrows():
        data = row.to_numpy()
        interpolated_values = get_interpolated(list(data))
        interpolated_CDR_data = interpolated_CDR_data.append(pd.Series(interpolated_values, index=new_columns), ignore_index=True)

    Demand_data= {}
    Supply_data={}
    Carbon_Target={}
    CDR_cost={}
    k=0
    for j in ['M','H','L']:
        Demand_data[j]={}
        Supply_data[j]={}
        Carbon_Target[j]={}
        CDR_cost[j]={}
        for t in years:
            Demand_data[j][t]={}
            Supply_data[j][t]={}
            a=0
            for ds in ['AMM','METH','Oil']:
                Demand_data[j][t][ds]={}
                Supply_data[j][t][ds]={}
                value1 = interpolated_demand_data.at[3*k+a,str(t)]*1000000000
                for p in province:
                    Demand_data[j][t][ds][p]=value1*province_share.loc[province_share['PROVINCE'] == p, '{}_Share'.format(ds)].tolist()[0]
                    Supply_data[j][t][ds][p]=value1*province_share.loc[province_share['PROVINCE'] == p, '{}_Supply_Share'.format(ds)].tolist()[0]
                a=a+1
            value3=interpolated_CP_data.at[k,str(t)]
            Carbon_Target[j][t]=value3
            value4=interpolated_CDR_data.at[k,str(t)]
            CDR_cost[j][t]=value4
        k=k+1
    k=1
    for j in ['h','l']:
        CDR_cost[j]={}
        for t in years:
            value4=(interpolated_CDR_data.at[0,str(t)]*0.5+interpolated_CDR_data.at[k,str(t)]*0.5)
            CDR_cost[j][t]=value4
        k=k+1
    return Demand_data,Supply_data,Carbon_Target,CDR_cost



## EXP chemical ratio and demand ratio

In [22]:
def get_two_ratio_2020_no_P():
    data=np.array(pd.read_excel('data.xlsx',sheet_name='H2_P&C',index_col=None).iloc[:, 1:])
    matrix_by_row = data / data.sum(axis=1, keepdims=True)
    matrix_by_col = data / data.sum(axis=0, keepdims=True)

    product=['AMM','METH','Oil']
    process=['CG','SMR','EXT']

    EXP_chemical_ratio={}
    for i in process:
        EXP_chemical_ratio[i]={}
    chemical_EXP_ratio={}
    for i in product:
        chemical_EXP_ratio[i]={}

    for i in range(3):
        for j in range(3):
            EXP_chemical_ratio[process[j]][product[i]]=matrix_by_col[i][j]
            chemical_EXP_ratio[product[i]][process[j]]=matrix_by_row[i][j]

    return EXP_chemical_ratio,chemical_EXP_ratio



In [23]:
def get_two_ratio_2020():
    EXP_chemical_ratio_np,chemical_EXP_ratio_np=get_two_ratio_2020_no_P()
    PS=pd.read_excel('data.xlsx',sheet_name='province_structure')
    chemical_EXP_ratio={}
    for p in province:
        chemical_EXP_ratio[p]={}
        for c in chemicals:
            chemical_EXP_ratio[p][c]={}
            for j in ['CG','SMR','EXT']:
                if c in ['AMM','METH']:
                    chemical_EXP_ratio[p][c][j]=   PS.loc[PS['PROVINCE'] == p, '{}-{}'.format(c,j)].tolist()[0] /(0.00000000000001+PS.loc[PS['PROVINCE'] == p, '{}-CG'.format(c)].tolist()[0]+PS.loc[PS['PROVINCE'] == p, '{}-SMR'.format(c)].tolist()[0]+PS.loc[PS['PROVINCE'] == p,'{}-EXT'.format(c)].tolist()[0])
                elif c=='Oil':
                    chemical_EXP_ratio[p][c][j]=chemical_EXP_ratio_np[c][j]
        for hvc in ['ETH','PRO','BTX']:
            chemical_EXP_ratio[p][hvc]={}
            for hvc_process in HVC_relations[hvc]:
                chemical_EXP_ratio[p][hvc][hvc+'-'+hvc_process]=(PS.loc[PS['PROVINCE'] == p, '{}'.format(hvc+'-'+hvc_process)].tolist()[0])/(0.000000001+sum(PS.loc[PS['PROVINCE'] == p, '{}'.format(hvc+'-'+hvc_p)].tolist()[0] for hvc_p in HVC_relations[hvc] ))
                
            
    EXP_chemical_ratio={}
    for j in ['CG','SMR','EXT']:
        EXP_chemical_ratio[j]={}
        for c in chemicals:
            if c in['AMM','METH']:
                EXP_chemical_ratio[j][c]={}
                tot=PS['{}-{}'.format(c,j)].sum()
                EXP_chemical_ratio[j][c]['ori']=EXP_chemical_ratio_np[j][c]
                for p in province:
                    EXP_chemical_ratio[j][c][p]=EXP_chemical_ratio_np[j][c]*PS.loc[PS['PROVINCE'] == p, '{}-{}'.format(c,j)].tolist()[0]/tot
            elif c=='Oil':
                EXP_chemical_ratio[j][c]={}
                EXP_chemical_ratio[j][c]['ori']=EXP_chemical_ratio_np[j][c]
                for p in province:
                    EXP_chemical_ratio[j][c][p]=EXP_chemical_ratio_np[j][c]*PS.loc[PS['PROVINCE'] == p, 'Oil_Share'].tolist()[0]
    for process in HVC_processes:
        EXP_chemical_ratio[process]={}
        tot=PS[process].sum()
        EXP_chemical_ratio[process]['ori']=tot
        for p in province:
            EXP_chemical_ratio[process][p]=PS.loc[PS['PROVINCE'] == p, process].tolist()[0]/tot
        
    return EXP_chemical_ratio,chemical_EXP_ratio

In [24]:
def get_trans_costXdistance(d,c,trans_cost):
    if d==0:
        return 0
    if c=='Oil':
        c='H2'
    return trans_cost[c][int(d//500*500)+500]/1000*7#to kg, then to CNY

In [25]:
EXP_chemical_ratio,chemical_EXP_ratio=get_two_ratio_2020()
loss=0.05
H2_demand_ratio={
    'AMM':0.17*(1+loss),
    'METH_CO_based':0.12*(1+loss),
    'METH_CO2_based':0.18*(1+loss),
    'Oil':0.012*(1+loss)
}
CC_ratio=0.109#t CO2/t Oil
NSC_dict={
    'ETH':{'Low':0.4,'High':0.55},
    'PRO':{'Low':0.2,'High':0.3},
    'BTX':{'Low':0.2,'High':0.3},
}

In [26]:
inter=5
years=list(range(2020,2061,inter))

In [38]:
def solve_model(
    CAP, OM, FS, NC, LT, OH_max, oh_process_max, HP, CF, CF_METH,
    CCS_delta_CF, CCS_delta_CF_METH, EXCAP, EXCAP_process, POT, WS_POT,
    CO2_POT, CO2_level, CCS_cost, trans_cost, HVC_para, demand_all,
    DS_ratio, EI, ele_data, Heat_para, SCE_data, S, store_name,
    GR_upper=0, GR_low_5_delta=0
):
    R = 0.05
    model = gp.Model()

    # ======================================================================
    # Decision variables (independent)
    # ======================================================================
    # x: new process capacity additions (kW)
    # y: process operation scale (kWh)
    # oh: operating hours (h/year) - implied via OH_max constraints
    x = model.addVars(
        years, technologies, processes, chemicals, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='process construction amount'
    )
    y = model.addVars(
        years, processes, chemicals, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='process operation amount'
    )

    # x_HVC and y_HVC: HVC process additions and outputs (both in tonnes)
    # Note: most of this codebase uses kg, but HVC here is in tonnes.
    x_HVC = model.addVars(
        years, HVCs, HVC_processes, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='process construction amount'
    )
    y_HVC = model.addVars(
        years, HVCs, HVC_processes, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='process operation amount'
    )

    # Heat/electricity/CCS for heat competition:
    # heat in GJ, electricity in MWh, CCS in kg CO2
    heat_output = model.addVars(
        years, ['Biomass', 'NG'], ['AMM', 'METH'] + HVCs, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name='heat'
    )
    ele_input = model.addVars(
        years, ['AMM', 'METH'] + HVCs, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name='heat'
    )
    CCS_heat = model.addVars(
        years, ['Biomass', 'NG'], ['AMM', 'METH'] + HVCs, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name='heat-CCS'
    )

    # ======================================================================
    # Dependent variables / auxiliaries
    # ======================================================================
    # U: total technology capacity (aggregated)
    U = model.addVars(
        years, technologies, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='capacity'
    )

    # u: new capacity for process (excluding legacy capacity; legacy added later via up)
    u = model.addVars(
        years, technologies, processes, chemicals, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='new capacity for process'
    )

    # u_HVC: capacity for HVC processes including legacy capacity (explicitly handled)
    u_HVC = model.addVars(
        years, HVCs, HVC_processes, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='new capacity for process'
    )

    # up: total available process capacity upper bound (legacy + new), used as cap limit
    up = model.addVars(
        years, processes, chemicals, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='process capacity max'
    )

    # X: new tech capacity additions (aggregated over processes)
    X = model.addVars(
        years, technologies, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='tech construction amount'
    )

    # Y: tech operation scale (aggregated kWh across associated processes)
    Y = model.addVars(
        years, technologies, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='technology operation amount'
    )

    # H2: hydrogen production from each process (kWh input * HP, unit consistent with HP)
    H2 = model.addVars(
        years, processes, chemicals, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='H2 production amount'
    )

    # CO2 and CO2_METH: auxiliary emissions variables (sign conventions adjusted later)
    CO2 = model.addVars(
        years, processes, ['AMM', 'Oil'], province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='CO2 emission amount'
    )
    CO2_METH = model.addVars(
        years, METH_processes, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='CO2 emission amount METH'
    )

    # Cracking/refinery-related emissions and CCS
    CO2_cc = model.addVars(
        years, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='CO2_cc amount'
    )
    CCS_cc = model.addVars(
        years, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='CCS_cc amount'
    )

    # HVC process CCS capture (kg CO2)
    CCS_HVC = model.addVars(
        years, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='CCS_hvc amount'
    )

    # CCS amount (process-level), CDR amount (national)
    CCS_amount = model.addVars(
        years, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='CCS amount'
    )
    CDR_amount = model.addVars(
        years,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='CDR amount'
    )

    # Transportation flows between neighboring provinces
    trans_couple = model.addVars(
        [(t, c, p1, p2) for t in years for c in chemicals
         for p1 in province for p2 in adjacency_dict[p1]],
        lb=0, name="trans_couple"
    )

    # ======================================================================
    # Cost bookkeeping variables
    # ======================================================================
    cost_by_year = model.addVars(
        years, costs, chemicals, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name='Cost'
    )
    # Annualized capital cost paid per year by technology
    cost_by_tech = model.addVars(
        years, technologies, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name='Cost'
    )

    next_cost_by_year = model.addVars(
        years, ['AMM', 'METH'], province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS
    )

    # Methanol processes hydrogen demand
    H2_demand_METH = model.addVars(
        years, METH_processes, province,
        lb=0, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,
        name='METHANOL processes hydrogen demand'
    )

    # ======================================================================
    # Objective: minimize discounted total system cost
    # ======================================================================
    model.setObjective(
        gp.quicksum([
            1 / (1 + R) ** (t - 2020) * (
                # ----------------------------------------------------------
                # AMM / METH / Refinery-H2: CAPEX + OM + Existing + FS
                # ----------------------------------------------------------
                gp.quicksum([
                    CAP[tau][j] * X[tau, j, p]
                    for j in technologies
                    for tau in years if tau <= t and tau + LT[tau][j] > t
                    for p in province
                ]) +
                gp.quicksum([
                    OM[tau][j] * X[tau, j, p]
                    for j in technologies
                    for tau in years if tau <= t and tau + LT[tau][j] > t
                    for p in province
                ]) +
                gp.quicksum([(OM[2020][j] + CAP[2020][j]) * EXCAP[t][j] for j in technologies]) +
                gp.quicksum([FS[t][i] * y[t, i, c, p] for i in processes for c in chemicals for p in province]) +

                # ----------------------------------------------------------
                # HVC: CAPEX + OM + Existing + FS (MTO handled separately)
                # ----------------------------------------------------------
                gp.quicksum([
                    HVC_para[hvc][hvc + '-' + hvc_process]['CAP'] *
                    x_HVC[tau, hvc, hvc + '-' + hvc_process, p]
                    for hvc in HVCs for hvc_process in HVC_relations[hvc]
                    for tau in years if tau <= t and tau + LT[tau][hvc + '-' + hvc_process] > t
                    for p in province
                ]) +
                gp.quicksum([
                    HVC_para[hvc][hvc + '-' + hvc_process]['OM'] *
                    x_HVC[tau, hvc, hvc + '-' + hvc_process, p]
                    for hvc in HVCs for hvc_process in HVC_relations[hvc]
                    for tau in years if tau <= t and tau + LT[tau][hvc + '-' + hvc_process] > t
                    for p in province
                ]) +
                gp.quicksum([
                    (HVC_para[hvc][hvc + '-' + hvc_process]['CAP'] +
                     HVC_para[hvc][hvc + '-' + hvc_process]['OM']) *
                    EXCAP[t][hvc + '-' + hvc_process]
                    for hvc in HVCs for hvc_process in HVC_relations[hvc]
                ]) +
                gp.quicksum([
                    HVC_para[hvc][hvc + '-' + hvc_process]['FS-cost'][t] /
                    HVC_para[hvc][hvc + '-' + hvc_process]['out/in'][t] *
                    y_HVC[t, hvc, hvc + '-' + hvc_process, p]
                    for hvc in HVCs for hvc_process in HVC_relations[hvc]
                    if hvc_process != 'MTO'
                    for p in province
                ]) +
                # Methanol import cost for MTO (tonnes; Mt -> t conversion)
                gp.quicksum([
                    HVC_para['ETH']['ETH-MTO']['FS-cost'][t] *
                    demand_all['METH']['MTO_import'][t] * 1000000
                ]) +

                # ----------------------------------------------------------
                # Heat and electricity supply costs
                # ----------------------------------------------------------
                gp.quicksum([
                    heat_output[t, f, c, p] * (
                        Heat_para[f]['CAP+OM'] +
                        1 / Heat_para[f]['heat eff'] / Heat_para[f]['heat value'] * Heat_para[f]['FS'][t]
                    )
                    for f in ['Biomass', 'NG']
                    for c in ['AMM', 'METH'] + HVCs
                    for p in province
                ]) +
                gp.quicksum([
                    ele_input[t, c, p] * 1000 * ele_data['cost'][p][t]
                    for c in ['AMM', 'METH'] + HVCs
                    for p in province
                ]) +

                # ----------------------------------------------------------
                # CDR and CCS costs
                # ----------------------------------------------------------
                gp.quicksum([CDR_amount[t] * SCE_data['CDR_cost'][t] * 0.001]) +
                gp.quicksum([
                    CCS_cost[p][t] * 0.001 * y[t, i, c, p] * HP[t][i] * CCS_delta_CF[t][i]
                    for c in ['AMM', 'Oil'] for p in province
                    for i in processes if (i == 'BG-CCS' or i == 'CG-CCS' or i == 'SMR-CCS')
                ]) +
                gp.quicksum([
                    CCS_cost[p][t] * 0.001 * H2_demand_METH[t, j, p] / 0.128 * CCS_delta_CF_METH[t][j]
                    for p in province for j in ['Coal-based-CCS', 'Gas-based-CCS', 'Biomass-based-CCS']
                ]) +
                gp.quicksum([
                    CCS_cost[p][t] * 0.001 * (
                        CCS_cc[t, p] + CCS_HVC[t, p] +
                        sum(CCS_heat[t, f, c, p] for f in ['Biomass', 'NG'] for c in ['AMM', 'METH'] + HVCs)
                    )
                    for p in province
                ]) +

                # ----------------------------------------------------------
                # "Next" (downstream) cost components for AMM/METH
                # ----------------------------------------------------------
                gp.quicksum([y[t, i, 'AMM', p] * HP[t][i] for i in processes for p in province]) /
                H2_demand_ratio['AMM'] * (NC[t]['AMM']['CAP'] + NC[t]['AMM']['OM'] + NC[t]['AMM']['FS']) +

                gp.quicksum([H2_demand_METH[t, k, p] for k in METH_processes[:-2] for p in province]) /
                H2_demand_ratio['METH_CO_based'] * (NC[t]['METH1']['CAP'] + NC[t]['METH1']['OM'] + NC[t]['METH1']['FS']) +
                gp.quicksum([H2_demand_METH[t, METH_processes[-2], p] for p in province]) /
                H2_demand_ratio['METH_CO2_based'] * (NC[t]['METH2']['CAP'] + NC[t]['METH2']['OM'] + NC[t]['METH2']['FS']) +
                gp.quicksum([H2_demand_METH[t, METH_processes[-1], p] for p in province]) /
                H2_demand_ratio['METH_CO2_based'] * (NC[t]['METH3']['CAP'] + NC[t]['METH3']['OM'] + NC[t]['METH3']['FS']) +

                # ----------------------------------------------------------
                # Transportation costs
                # ----------------------------------------------------------
                gp.quicksum(
                    trans_couple[t, c, p1, p2] *
                    get_trans_costXdistance(
                        df_distance[df_distance["Province"] == p1][p2].values[0],
                        c, trans_cost
                    )
                    for c in chemicals
                    for p1 in province for p2 in adjacency_dict[p1]
                )
            )
            for t in years
        ]),
        gp.GRB.MINIMIZE
    )

    # ======================================================================
    # Cost accounting constraints
    # ======================================================================
    model.addConstrs(
        cost_by_year[t, 'CAP', c, p] == (
            gp.quicksum([
                CAP[tau][j] * x[tau, j, i, c, p]
                for i in processes for j in technologies
                for tau in years if tau <= t and tau + LT[tau][j] > t
            ]) +
            gp.quicksum([
                CAP[2020][j] * EXCAP[t][j] * EXP_chemical_ratio[j][c][p]
                for j in ['CG', 'SMR', 'EXT']
            ])
        )
        for p in province for t in years for c in chemicals
    )

    # Annualized CAPEX by technology (paid each year)
    model.addConstrs(
        cost_by_tech[t, j, p] == (
            gp.quicksum([
                CAP[tau][j] * x[tau, j, i, c, p]
                for i in processes for c in chemicals
                for tau in years if tau <= t and tau + LT[tau][j] > t
            ]) +
            gp.quicksum([
                CAP[2020][j] * EXCAP[t][j] * EXP_chemical_ratio[j][c][p]
                for c in chemicals if j in ['CG', 'SMR', 'EXT']
            ])
        )
        for p in province for t in years for j in technologies
    )

    model.addConstrs(
        cost_by_year[t, 'OM', c, p] == (
            gp.quicksum([
                OM[tau][j] * x[tau, j, i, c, p]
                for i in processes for j in technologies
                for tau in years if tau <= t and tau + LT[tau][j] > t
            ]) +
            gp.quicksum([
                OM[2020][j] * EXCAP[t][j] * EXP_chemical_ratio[j][c][p]
                for j in ['CG', 'SMR', 'EXT']
            ])
        )
        for p in province for t in years for c in chemicals
    )

    model.addConstrs(
        cost_by_year[t, 'FS', c, p] == gp.quicksum([FS[t][i] * y[t, i, c, p] for i in processes])
        for p in province for t in years for c in chemicals
    )

    model.addConstrs(
        cost_by_year[t, 'CCS', c, p] == gp.quicksum([
            CCS_cost[p][t] * 0.001 * y[t, i, c, p] * HP[t][i] * CCS_delta_CF[t][i]
            for i in processes if (i == 'BG-CCS' or i == 'CG-CCS' or i == 'SMR-CCS')
        ])
        for c in ['AMM'] for t in years for p in province
    )

    model.addConstrs(
        cost_by_year[t, 'CCS', c, p] == (
            gp.quicksum([CCS_cost[p][t] * 0.001 * CCS_cc[t, p]]) +
            gp.quicksum([
                CCS_cost[p][t] * 0.001 * y[t, i, c, p] * HP[t][i] * CCS_delta_CF[t][i]
                for i in processes if (i == 'BG-CCS' or i == 'CG-CCS' or i == 'SMR-CCS')
            ])
        )
        for c in ['Oil'] for t in years for p in province
    )

    model.addConstrs(
        cost_by_year[t, 'CCS', 'METH', p] == gp.quicksum([
            CCS_cost[p][t] * 0.001 * H2_demand_METH[t, j, p] / 0.11 * CCS_delta_CF_METH[t][j]
            for j in ['Coal-based-CCS', 'Gas-based-CCS', 'Biomass-based-CCS']
        ])
        for t in years for p in province
    )

    model.addConstrs(
        next_cost_by_year[t, 'AMM', p] == (
            gp.quicksum([y[t, i, 'AMM', p] * HP[t][i] for i in processes]) /
            H2_demand_ratio['AMM'] * (NC[t]['AMM']['CAP'] + NC[t]['AMM']['OM'] + NC[t]['AMM']['FS'])
        )
        for t in years for p in province
    )

    model.addConstrs(
        next_cost_by_year[t, 'METH', p] == (
            gp.quicksum([H2_demand_METH[t, k, p] for k in METH_processes[:-2]]) /
            H2_demand_ratio['METH_CO_based'] * (NC[t]['METH1']['CAP'] + NC[t]['METH1']['OM'] + NC[t]['METH1']['FS']) +
            H2_demand_METH[t, METH_processes[-2], p] /
            H2_demand_ratio['METH_CO2_based'] * (NC[t]['METH2']['CAP'] + NC[t]['METH2']['OM'] + NC[t]['METH2']['FS']) +
            H2_demand_METH[t, METH_processes[-1], p] /
            H2_demand_ratio['METH_CO2_based'] * (NC[t]['METH3']['CAP'] + NC[t]['METH3']['OM'] + NC[t]['METH3']['FS'])
        )
        for p in province for t in years
    )

    # ======================================================================
    # Definitions: H2 and CO2 accounting
    # ======================================================================
    # H2 definition: operation * hydrogen productivity
    model.addConstrs(
        (H2[t, i, c, p] == y[t, i, c, p] * HP[t][i])
        for p in province for i in processes for c in chemicals for t in years
    )

    # CO2 definition: similar to H2, but convert negative CF to positive via abs()
    model.addConstrs(
        (CO2[t, i, c, p] == y[t, i, c, p] * HP[t][i] * np.abs(CF[t][i]))
        for p in province for c in ['AMM', 'Oil'] for i in processes for t in years
    )

    # Methanol CO2: handle sign conventions (biomass/DAC etc.)
    model.addConstrs(
        (CO2_METH[t, i, p] == -H2_demand_METH[t, i, p] / H2_demand_ratio['METH_CO_based'] * CF_METH[t][i])
        for p in province for i in ['Biomass-based', 'Biomass-based-CCS'] for t in years
    )
    model.addConstrs(
        (CO2_METH[t, i, p] == H2_demand_METH[t, i, p] / H2_demand_ratio['METH_CO2_based'] * CF_METH[t][i])
        for p in province for i in ['CO2-GH-based', 'DAC-GH-based'] for t in years
    )
    model.addConstrs(
        (CO2_METH[t, i, p] == H2_demand_METH[t, i, p] / H2_demand_ratio['METH_CO_based'] * CF_METH[t][i])
        for p in province for i in METH_processes[0:6] for t in years
    )

    # ======================================================================
    # Definitions: technology capacity aggregation U and process capacity u/u_HVC
    # ======================================================================
    # U definition: actual installed capacity (including legacy for certain technologies)
    model.addConstrs(
        (
            (
                gp.quicksum(EXCAP[t][j] * EXP_chemical_ratio[j][c][p] for c in chemicals) +
                gp.quicksum(X[tau, j, p] for tau in years if tau <= t and tau + LT[tau][j] > t)
            ) == U[t, j, p]
            if j in ['CG', 'SMR', 'EXT'] else
            gp.quicksum(X[tau, j, p] for tau in years if tau <= t and tau + LT[tau][j] > t) == U[t, j, p]
        )
        for j in technologies for t in years for p in province
    )

    # HVC capacity u_HVC = existing + in-service x_HVC (all in tonnes)
    model.addConstrs(
        u_HVC[t, hvc, hvc + '-' + hvc_process, p] ==
        gp.quicksum(
            x_HVC[tau, hvc, hvc + '-' + hvc_process, p]
            for tau in years if tau <= t and tau + LT[tau][hvc + '-' + hvc_process] > t
        ) +
        EXCAP[t][hvc + '-' + hvc_process] * EXP_chemical_ratio[hvc + '-' + hvc_process][p] * 1000000
        for hvc in HVCs for hvc_process in HVC_relations[hvc] for t in years for p in province
    )

    # Enforce invalid tech-to-process assignments to be zero
    model.addConstrs(
        x[t, j, i, c, p] == 0
        for p in province for i in processes for j in technologies for t in years for c in chemicals
        if i not in TP_relation[j]
    )

    # Force CCS process "new x" to zero (keeps u definition unified downstream)
    model.addConstrs(
        x[t, j, i, c, p] == 0
        for p in province for i in ['CG-CCS', 'BG-CCS', 'SMR-CCS']
        for j in PT_relation[i] for t in years for c in chemicals
    )

    # u definitions: for EXT/CG/SMR include legacy EXCAP shares; other processes are new-only
    model.addConstrs(
        (
            u[t, j, i, c, p] ==
            EXCAP[t][j] * EXP_chemical_ratio[j][c][p] +
            gp.quicksum(x[tau, j, i, c, p] for tau in years if tau <= t and tau + LT[tau][j] > t)
        )
        for i in ['EXT', 'CG', 'SMR'] for j in PT_relation[i]
        for t in years for c in chemicals for p in province
    )
    model.addConstrs(
        (
            u[t, j, i, c, p] ==
            gp.quicksum(x[tau, j, i, c, p] for tau in years if tau <= t and tau + LT[tau][j] > t)
        )
        for i in ['Grid-AE', 'Grid-PEM', 'Nuclear-THP', 'BG'] for j in PT_relation[i]
        for t in years for c in chemicals for p in province
    )

    # Nuclear hydrogen (no storage)
    model.addConstrs(
        (
            u[t, j, i, c, p] ==
            gp.quicksum(x[tau, j, i, c, p] for tau in years if tau <= t and tau + LT[tau][j] > t)
        )
        for i in ['Nuclear-wes-AE', 'Nuclear-wes-PEM', 'Nuclear-wes-SOEC']
        for j in PT_relation[i] for t in years for c in chemicals for p in province
    )

    # Technologies with optional storage: add electrolyzer capacity + storage-linked capacity
    model.addConstrs(
        (u[t, j, i, c, p]) ==
        gp.quicksum(x[tau, j, i, c, p] for tau in years if tau <= t and tau + LT[tau][j] > t) +
        gp.quicksum(x[tau, j, es_index[i], c, p] for tau in years if tau <= t and tau + LT[tau][j] > t)
        for i in list(es_index.keys()) for j in PT_relation[i]
        for t in years for c in chemicals for p in province
    )
    model.addConstrs(
        u[t, j, i, c, p] == u[t, j, es_index[i], c, p]
        for i in list(es_index.keys()) for j in PT_relation[i]
        for t in years for c in chemicals for p in province
    )

    # Storage capacity construction
    model.addConstrs(
        (
            u[t, 'Energy_stroage', i, c, p] ==
            gp.quicksum(
                x[tau, 'Energy_stroage', i, c, p]
                for tau in years if tau <= t and tau + LT[tau]['Energy_stroage'] > t
            )
        )
        for i in list(es_index.values()) for t in years for c in chemicals for p in province
    )

    # ======================================================================
    # Aggregation: X and Y
    # ======================================================================
    # X: sum of new capacity serving all processes and chemicals
    model.addConstrs(
        (X[t, j, p] == gp.quicksum(x[t, j, i, c, p] for i in processes for c in chemicals))
        for p in province for j in technologies for t in years
    )

    # Y: sum of all process operation (kWh) using technology j
    model.addConstrs(
        (Y[t, j, p] == gp.quicksum(y[t, i, c, p] for i in TP_relation[j] for c in chemicals))
        for p in province for j in technologies for t in years
    )

    # ======================================================================
    # H2_demand_METH definitions
    # ======================================================================
    # One-to-one dedicated supply groups
    model.addConstrs(
        (H2_demand_METH[t, k, p] == gp.quicksum(H2[t, i, 'METH', p] for i in PP_relation[k]))
        for p in province for t in years
        for k in ['Coal-based-CCS', 'Gas-based', 'Gas-based-CCS', 'By-product-based', 'Biomass-based', 'Biomass-based-CCS']
    )

    # Green hydrogen total consumption (industrial CO2 + DAC CO2 + 0.5 share from coal-green coupling)
    model.addConstrs(
        (
            H2_demand_METH[t, 'CO2-GH-based', p] +
            H2_demand_METH[t, 'DAC-GH-based', p] +
            H2_demand_METH[t, 'Coal-GH-based', p] * 0.5
            ==
            gp.quicksum(H2[t, i, 'METH', p] for i in PP_relation['CO2-GH-based'])
        )
        for t in years for p in province
    )

    # Coal-related H2 equals coal methanol + coal-green hybrid share
    model.addConstrs(
        (H2_demand_METH[t, 'Coal-based', p] + 0.5 * H2_demand_METH[t, 'Coal-GH-based', p] == H2[t, 'CG', 'METH', p])
        for t in years for p in province
    )

    # ======================================================================
    # Capacity constraints
    # ======================================================================
    # Technology capacity utilization: operation <= capacity * operating hours
    model.addConstrs(
        (Y[t, j, p] <= U[t, j, p] * OH_max[t][j][p])
        for p in province for j in technologies for t in years
    )

    # HVC capacity constraint
    model.addConstrs(
        y_HVC[t, hvc, hvc + '-' + hvc_process, p] <= u_HVC[t, hvc, hvc + '-' + hvc_process, p]
        for hvc in HVCs for hvc_process in HVC_relations[hvc]
        for t in years for p in province
    )

    # For CCS-coupled processes: direct operation + CCS operation <= base process capacity
    model.addConstrs(
        (y[t, i, c, p] + y[t, CCS_index[i], c, p] <= u[t, i, i, c, p] * OH_max[t][i][p])
        for i in list(CCS_index.keys()) for p in province for t in years for c in chemicals
    )

    # Full one-step conversion processes (Grid/THP/EXT): bound by capacity
    model.addConstrs(
        (
            y[t, i, c, p] <= u[t, PT_relation[i][0], i, c, p] * OH_max[t][PT_relation[i][0]][p]
            for i in ['Grid-AE', 'Grid-PEM', 'Nuclear-THP', 'EXT']
            for p in province for t in years for c in chemicals
        ),
        name='capacity upper'
    )

    # Nuclear hydrogen without storage
    model.addConstrs(
        y[t, i, c, p] <= u[t, j, i, c, p] * OH_max[t][j][p]
        for i in ['Nuclear-wes-AE', 'Nuclear-wes-PEM', 'Nuclear-wes-SOEC']
        for p in province for j in PT_relation[i] for t in years for c in chemicals
    )

    # With potential storage: electrolyzer + storage operation <= generation equipment capability
    model.addConstrs(
        (y[t, i, c, p] + y[t, es_index[i], c, p] <= u[t, j, i, c, p] * OH_max[t][j][p])
        for i in list(es_index.keys()) for j in PT_relation[i]
        for t in years for p in province for c in chemicals
    )

    # Without storage: electrolyzer max utilization limited by generation equipment
    model.addConstrs(
        (y[t, i, c, p] <= u[t, j, i, c, p] * OH_max[t][PT_relation[i][0]][p])
        for i in list(es_index.keys()) for j in ['AE', 'PEM']
        for p in province for t in years for c in chemicals
    )

    # Storage device operation must satisfy its own availability
    model.addConstrs(
        (y[t, i, c, p] <= u[t, 'Energy_stroage', i, c, p] * OH_max[t]['Energy_stroage'][p])
        for i in list(es_index.values()) for t in years for c in chemicals for p in province
    )

    # ======================================================================
    # Resource potential constraints
    # ======================================================================
    model.addConstrs(
        (U[t, j, p] <= POT[t][j][p] for j in technologies if (j in ['Hydro']) for t in years for p in province),
        name='Hydro_POT'
    )
    model.addConstrs(
        (U[t, j, p] <= POT[t][j][p] for j in technologies if (j in ['SMR']) for t in years for p in province),
        name='POT'
    )

    model.addConstrs(
        (
            u[t, 'EXT', 'EXT', c, p] <=
            POT[t]['EXT'][p] * (
                EXP_chemical_ratio['EXT'][c][p] /
                (0.0000001 + EXP_chemical_ratio['EXT']['AMM'][p] + EXP_chemical_ratio['EXT']['METH'][p] + EXP_chemical_ratio['EXT']['Oil'][p])
            )
            for c in ['AMM', 'METH'] for t in years for p in province
        ),
        name='EXT'
    )

    model.addConstrs(
        (
            u[t, 'EXT', 'EXT', 'Oil', p] <=
            (
                POT[t]['EXT'][p] +
                7000000 * (POT[2020]['EXT'][p] / sum(POT[2020]['EXT'][p] for p in province)) *
                (demand_all['Oil']['Tot'][t] * DS_ratio['Supply_ratio'][p]['Oil_Supply_Share']) /
                (demand_all['Oil']['Tot'][2020] * DS_ratio['Supply_ratio'][p]['Oil_Supply_Share'] + 0.000001) +
                2500000 * (y_HVC[t, 'PRO', 'PRO-PDH', p] / (demand_all['PRO']['Tot'][2020] * 0.35 * 1000000))
            ) * (
                EXP_chemical_ratio['EXT']['Oil'][p] /
                (0.000000001 + sum(EXP_chemical_ratio['EXT'][c][p] for c in ['AMM', 'METH', 'Oil']))
            )
            for t in years for p in province
        ),
        name='EXT-Oil'
    )

    # Industrial point-source CO2 supply is disabled (CO2 POT set to 0)
    model.addConstrs(
        ((H2_demand_METH[t, 'CO2-GH-based', p] == 0) for t in years for p in province),
        name='industrial CO2 supply'
    )

    model.addConstrs(
        (
            gp.quicksum([y[t, i, c, p] for i in ['BG', 'BG-CCS'] for c in chemicals]) * 3.6 * 0.000000000001 +
            gp.quicksum([heat_output[t, 'Biomass', c, p] / Heat_para['Biomass']['heat eff'] / 1000000000 for c in ['AMM', 'METH'] + HVCs])
            <= SCE_data['Bio_POT'][p]
            for t in years for p in province
        ),
        name='Biomass_POT'
    )

    # Wind & solar bounded to 20% of total potential (Mt -> t scaling embedded)
    model.addConstrs(
        (U[t, j, p] <= 0.2 * WS_POT[j][p] * 1000000 for t in years for p in province for j in ['Wind', 'Solar']),
        name='W&S POT'
    )

    # ======================================================================
    # Refinery-related HVC process bounds linked to oil demand
    # ======================================================================
    model.addConstrs(
        u_HVC[t, hvc_process[0:3], hvc_process, p] <=
        u_HVC[2020, hvc_process[0:3], hvc_process, p] *
        demand_all['Oil']['Tot'][t] / demand_all['Oil']['Tot'][2020]
        for t in years[1:] for p in province for hvc_process in ['PRO-CC', 'BTX-CR']
    )
    model.addConstrs(
        y_HVC[t, hvc_process[0:3], hvc_process, p] <=
        y_HVC[2020, hvc_process[0:3], hvc_process, p] *
        demand_all['Oil']['Tot'][t] / demand_all['Oil']['Tot'][2020]
        for t in years[1:] for p in province for hvc_process in ['PRO-CC', 'BTX-CR']
    )

    # ======================================================================
    # NSC structure constraints
    # ======================================================================
    # NSC must satisfy composition bounds:
    # Total = 60; ETH share: 40-60% of 60; PRO: 20-32%; BTX: 16-30%
    model.addConstrs(
        (
            gp.quicksum([y_HVC[t, hvc, hvc + '-NSC', p] for p in province]) <=
            NSC_dict[hvc]['High'] * gp.quicksum([y_HVC[t, hvc, hvc + '-NSC', p] for hvc in HVCs for p in province])
            for hvc in HVCs for t in years
        ),
        name='NSC balance'
    )
    model.addConstrs(
        (
            gp.quicksum([y_HVC[t, hvc, hvc + '-NSC', p] for p in province]) >=
            NSC_dict[hvc]['Low'] * gp.quicksum([y_HVC[t, hvc, hvc + '-NSC', p] for hvc in HVCs for p in province])
            for hvc in HVCs for t in years
        ),
        name='NSC balance'
    )

    # ======================================================================
    # Smoothness constraints: avoid radical regional industrial shifts
    # ======================================================================
    model.addConstrs(
        (
            gp.quicksum([H2[t, i, c, p] for i in processes]) <=
            600000 / 5 * inter + (1 + 0.3 / 5 * inter) * gp.quicksum([H2[t - inter, i, c, p] for i in processes])
            for t in years if t >= (2020 + inter) for c in chemicals for p in province
        ),
        name='Prevent radical industrial changes'
    )
    model.addConstrs(
        (
            gp.quicksum([H2[t, i, c, p] for i in processes]) >=
            (1 - 0.25 / 5 * inter) * gp.quicksum([H2[t - inter, i, c, p] for i in processes])
            for t in years if t >= (2020 + inter) for c in chemicals for p in province
        ),
        name='Prevent radical industrial changes'
    )
    model.addConstrs(
        (
            gp.quicksum([H2[t, i, c, p] for c in chemicals]) >=
            (1 - 0.3 / 5 * inter) * gp.quicksum([H2[t - inter, i, c, p] for c in chemicals])
            for t in years if (t >= (2020 + inter)) & (t < 2045) for i in processes for p in province
        ),
        name='Prevent radical industrial changes'
    )

    model.addConstrs(
        (
            y_HVC[t, hvc, hvc + '-' + hvc_process, p] <=
            100000 / 5 * inter + (1 + 0.3 / 5 * inter) * y_HVC[t - inter, hvc, hvc + '-' + hvc_process, p]
            for t in years if t >= (2020 + inter)
            for hvc in HVCs for hvc_process in HVC_relations[hvc] for p in province
        ),
        name='Prevent radical industrial changes'
    )
    model.addConstrs(
        (
            y_HVC[t, hvc, hvc + '-' + hvc_process, p] >=
            (1 - 0.2 / 5 * inter) * y_HVC[t - inter, hvc, hvc + '-' + hvc_process, p]
            for t in years if t >= (2020 + inter)
            for hvc in HVCs for hvc_process in HVC_relations[hvc] for p in province
        ),
        name='Prevent radical industrial changes'
    )

    # ======================================================================
    # Import dependence / technology bounds
    # ======================================================================
    # Cap NSC/ESC/PDH: 30%/45% shares; NSC capacity <= 1.3x 2020
    model.addConstrs(
        gp.quicksum(y_HVC[t, 'ETH', 'ETH-ESC', p] for p in province) <=
        0.3 * gp.quicksum(y_HVC[t, 'ETH', 'ETH-' + hvc_process, p] for hvc_process in HVC_relations['ETH'] for p in province)
        for t in years
    )
    model.addConstrs(
        gp.quicksum(y_HVC[t, 'PRO', 'PRO-PDH', p] for p in province) <=
        0.45 * gp.quicksum(y_HVC[t, 'PRO', 'PRO-' + hvc_process, p] for hvc_process in HVC_relations['PRO'] for p in province)
        for t in years
    )
    model.addConstrs(
        gp.quicksum(u_HVC[t, 'ETH', 'ETH-NSC', p] for p in province) <=
        1.3 * gp.quicksum(u_HVC[2020, 'ETH', 'ETH-NSC', p] for p in province)
        for t in years[1:]
    )

    # ======================================================================
    # Nuclear hydrogen share limit (safety): <= 10% (implemented via potential * demand scaling)
    # ======================================================================
    model.addConstrs(
        (
            gp.quicksum([H2[t, i, c, p] for i in ['Nuclear-THP', 'Nuclear-wes-AE', 'Nuclear-wes-PEM', 'Nuclear-wes-SOEC'] for c in chemicals]) <=
            POT[t]['Nuclear'][p] / 100 * 0.25 * 1000000000 * sum([
                H2_demand_ratio['AMM'] * demand_all['AMM']['Tot'][t] +
                H2_demand_ratio['Oil'] * demand_all['Oil']['Tot'][t] +
                H2_demand_ratio['METH_CO_based'] * demand_all['METH']['Tot'][t]
            ])
            for t in years for p in province
        ),
        name='Nuclear_POT'
    )

    # ======================================================================
    # Technology balance constraints (AE vs PEM; Solar vs Wind after 2030)
    # ======================================================================
    model.addConstrs(
        (gp.quicksum([X[t, 'PEM', p] for p in province]) <= gp.quicksum([X[t, 'AE', p] for p in province]) * 1.1 for t in years),
        name='AE vs PEM'
    )
    model.addConstrs(
        (gp.quicksum([X[t, 'PEM', p] for p in province]) >= gp.quicksum([X[t, 'AE', p] for p in province]) * 0.7 for t in years),
        name='AE vs PEM'
    )
    model.addConstrs(
        (
            gp.quicksum([H2[t, i, c, p] for p in province for i in H2_groups['Solar'] for c in chemicals]) <=
            1.2 * gp.quicksum([H2[t, i, c, p] for p in province for i in H2_groups['Wind'] for c in chemicals])
            for t in years if t > 2030
        ),
        name='Solar vs Wind'
    )
    model.addConstrs(
        (
            gp.quicksum([H2[t, i, c, p] for p in province for i in H2_groups['Solar'] for c in chemicals]) >=
            0.8 * gp.quicksum([H2[t, i, c, p] for p in province for i in H2_groups['Wind'] for c in chemicals])
            for t in years if t > 2030
        ),
        name='Solar vs Wind'
    )

    # Biomass heat share <= 10% (quality constraints)
    model.addConstrs(
        (
            heat_output[t, 'Biomass', c, p] <=
            0.1 * gp.quicksum([heat_output[t, f, c, p] for f in ['Biomass', 'NG']])
            for c in ['AMM', 'METH'] + HVCs for t in years for p in province
        ),
        name='Bio vs NG'
    )

    # ======================================================================
    # Hydrogen balance & HVC balance (with inter-provincial transport)
    # ======================================================================
    model.addConstrs(
        (
            gp.quicksum([y[t, i, 'AMM', p] * HP[t][i] for i in processes]) / H2_demand_ratio['AMM'] +
            gp.quicksum(trans_couple[t, 'AMM', p1, p] for p1 in province if (t, 'AMM', p1, p) in trans_couple) -
            gp.quicksum(trans_couple[t, 'AMM', p, p2] for p2 in province if (t, 'AMM', p, p2) in trans_couple)
            >= demand_all['AMM']['Tot'][t] * 1000000000 * DS_ratio['Demand_ratio'][p]['AMM_Share']
            for t in years for p in province
        ),
        name='H2 balance'
    )

    model.addConstrs(
        (
            gp.quicksum([y[t, i, 'Oil', p] * HP[t][i] for i in processes]) / H2_demand_ratio['Oil'] +
            gp.quicksum(trans_couple[t, 'Oil', p1, p] for p1 in province if (t, 'Oil', p1, p) in trans_couple) / H2_demand_ratio['Oil'] -
            gp.quicksum(trans_couple[t, 'Oil', p, p2] for p2 in province if (t, 'Oil', p, p2) in trans_couple) / H2_demand_ratio['Oil']
            >= demand_all['Oil']['Tot'][t] * 1000000000 * DS_ratio['Demand_ratio'][p]['Oil_Share']
            for t in years for p in province
        ),
        name='H2 balance'
    )

    # METH balance: traditional uses - imports + domestic MTO + import MTO
    model.addConstrs(
        (
            gp.quicksum([H2_demand_METH[t, k, p] for k in METH_processes[:-2]]) / H2_demand_ratio['METH_CO_based'] +
            gp.quicksum([H2_demand_METH[t, k, p] for k in METH_processes[-2:]]) / H2_demand_ratio['METH_CO2_based'] +
            gp.quicksum(trans_couple[t, 'METH', p1, p] for p1 in province if (t, 'METH', p1, p) in trans_couple) -
            gp.quicksum(trans_couple[t, 'METH', p, p2] for p2 in province if (t, 'METH', p, p2) in trans_couple)
            >= (
                (demand_all['METH']['Traditional industry'][t] + demand_all['METH']['Energy carrier'][t]) *
                DS_ratio['Demand_ratio'][p]['METH_Share'] -
                demand_all['METH']['MTO_import'][t] * DS_ratio['Demand_ratio'][p]['METH_Import_Share']
            ) * 1000000000 +
            (y_HVC[t, 'ETH', 'ETH-MTO', p] + y_HVC[t, 'PRO', 'PRO-MTO', p]) / HVC_para['ETH']['ETH-MTO']['out/in'][t] * 1000
            for t in years for p in province
        ),
        name='METH balance'
    )

    model.addConstrs(
        (
            gp.quicksum(y_HVC[t, hvc, hvc + '-' + hvc_process, p] for hvc_process in HVC_relations[hvc]) >=
            demand_all[hvc]['Tot'][t] * DS_ratio['Demand_ratio'][p][f'{hvc}_Share'] * 1000000
            for hvc in HVCs for t in years for p in province
        ),
        name='HVC balance'
    )

    # ======================================================================
    # Heat and electricity balance
    # ======================================================================
    # Heat balance (GJ)
    model.addConstrs(
        (
            gp.quicksum([heat_output[t, f, 'AMM', p] for f in ['Biomass', 'NG']]) >=
            (gp.quicksum([y[t, i, 'AMM', p] * HP[t][i] for i in processes]) / H2_demand_ratio['AMM']) / 1000 * EI['AMM']['H_GJ/t']
            for t in years for p in province
        ),
        name='heat balance'
    )

    model.addConstrs(
        (
            gp.quicksum([heat_output[t, f, 'METH', p] for f in ['Biomass', 'NG']]) >=
            (
                gp.quicksum([H2_demand_METH[t, k, p] for k in METH_processes[:-2]]) / H2_demand_ratio['METH_CO_based'] +
                gp.quicksum([H2_demand_METH[t, k, p] for k in METH_processes[-2:]]) / H2_demand_ratio['METH_CO2_based']
            ) / 1000 * EI['METH']['H_GJ/t']
            for t in years for p in province
        ),
        name='heat balance'
    )

    model.addConstrs(
        (
            gp.quicksum([heat_output[t, f, hvc, p] for f in ['Biomass', 'NG']]) >=
            gp.quicksum([y_HVC[t, hvc, hvc + '-' + hvc_process, p] * EI[hvc + '-' + hvc_process]['H_GJ/t'] for hvc_process in HVC_relations[hvc]])
            for hvc in HVCs for t in years for p in province
        ),
        name='heat balance'
    )

    # Electricity balance (MWh)
    model.addConstrs(
        (
            ele_input[t, 'AMM', p] >=
            (gp.quicksum([y[t, i, 'AMM', p] * HP[t][i] for i in processes]) / H2_demand_ratio['AMM']) / 1000 * EI['AMM']['E_MWh/t']
            for t in years for p in province
        ),
        name='ele balance'
    )
    model.addConstrs(
        (
            ele_input[t, 'METH', p] >=
            (
                gp.quicksum([H2_demand_METH[t, k, p] for k in METH_processes[:-2]]) / H2_demand_ratio['METH_CO_based'] +
                gp.quicksum([H2_demand_METH[t, k, p] for k in METH_processes[-2:]]) / H2_demand_ratio['METH_CO2_based']
            ) / 1000 * EI['METH']['E_MWh/t']
            for t in years for p in province
        ),
        name='ele balance'
    )
    model.addConstrs(
        (
            ele_input[t, hvc, p] >=
            gp.quicksum([y_HVC[t, hvc, hvc + '-' + hvc_process, p] * EI[hvc + '-' + hvc_process]['E_MWh/t'] for hvc_process in HVC_relations[hvc]])
            for hvc in HVCs for t in years for p in province
        ),
        name='ele balance'
    )

    # ======================================================================
    # 2020 calibration constraints (match existing structure)
    # ======================================================================
    model.addConstrs(
        (
            H2[t, j, c, p] <=
            1.05 * H2_demand_ratio[c] * demand_all[c]['Tot'][t] * 1000000000 *
            DS_ratio['Supply_ratio'][p][f'{c}_Supply_Share'] * chemical_EXP_ratio[p][c][j]
            for t in [2020] for c in ['AMM', 'Oil'] for j in ['CG', 'SMR', 'EXT'] for p in province
        ),
        name='2020 cap'
    )
    model.addConstrs(
        (
            H2[t, j, c, p] >=
            0.95 * H2_demand_ratio[c] * demand_all[c]['Tot'][t] * 1000000000 *
            DS_ratio['Supply_ratio'][p][f'{c}_Supply_Share'] * chemical_EXP_ratio[p][c][j]
            for t in [2020] for c in ['AMM', 'Oil'] for j in ['CG', 'SMR', 'EXT'] for p in province
        ),
        name='2020 cap'
    )
    model.addConstrs(
        (
            H2[t, j, 'METH', p] <=
            1.05 * H2_demand_ratio['METH_CO_based'] * (
                (
                    (demand_all['METH']['Traditional industry'][t] + demand_all['METH']['Energy carrier'][t]) *
                    DS_ratio['Supply_ratio'][p]['METH_Supply_Share'] -
                    demand_all['METH']['MTO_import'][t] * DS_ratio['Demand_ratio'][p]['METH_Import_Share']
                ) * 1000000000 +
                (y_HVC[t, 'ETH', 'ETH-MTO', p] + y_HVC[t, 'PRO', 'PRO-MTO', p]) / HVC_para['ETH']['ETH-MTO']['out/in'][t] * 1000
            ) * chemical_EXP_ratio[p]['METH'][j]
            for t in [2020] for j in ['CG', 'SMR', 'EXT'] for p in province
        ),
        name='2020 cap'
    )
    model.addConstrs(
        (
            H2[t, j, 'METH', p] >=
            0.95 * H2_demand_ratio['METH_CO_based'] * (
                (
                    (demand_all['METH']['Traditional industry'][t] + demand_all['METH']['Energy carrier'][t]) *
                    DS_ratio['Supply_ratio'][p]['METH_Supply_Share'] -
                    demand_all['METH']['MTO_import'][t] * DS_ratio['Demand_ratio'][p]['METH_Import_Share']
                ) * 1000000000 +
                (y_HVC[t, 'ETH', 'ETH-MTO', p] + y_HVC[t, 'PRO', 'PRO-MTO', p]) / HVC_para['ETH']['ETH-MTO']['out/in'][t] * 1000
            ) * chemical_EXP_ratio[p]['METH'][j]
            for t in [2020] for j in ['CG', 'SMR', 'EXT'] for p in province
        ),
        name='2020 cap'
    )

    # Ban EXT additions before 2030
    model.addConstrs(X[t, 'EXT', p] == 0 for t in years if t <= 2030 for p in province)

    # HVC 2020 calibration
    model.addConstrs(
        (
            y_HVC[2020, hvc, hvc + '-' + hvc_process, p] <=
            1.05 * demand_all[hvc]['Tot'][2020] *
            DS_ratio['Supply_ratio'][p][f'{hvc}_Supply_Share'] *
            chemical_EXP_ratio[p][hvc][hvc + '-' + hvc_process] * 1000000
            for hvc in HVCs for hvc_process in HVC_relations[hvc] for p in province
        ),
        name='HVC 2020 cap'
    )
    model.addConstrs(
        (
            y_HVC[2020, hvc, hvc + '-' + hvc_process, p] >=
            0.95 * demand_all[hvc]['Tot'][2020] *
            DS_ratio['Supply_ratio'][p][f'{hvc}_Supply_Share'] *
            chemical_EXP_ratio[p][hvc][hvc + '-' + hvc_process] * 1000000
            for hvc in HVCs for hvc_process in HVC_relations[hvc] for p in province
        ),
        name='HVC 2020 cap'
    )

    # Ban biomass heat before 2030
    model.addConstrs(
        heat_output[t, 'Biomass', c, p] == 0
        for t in years if t <= 2030 for p in province for c in ['AMM', 'METH'] + HVCs
    )

    # ======================================================================
    # CO2 target constraint (system-wide accounting)
    # ======================================================================
    model.addConstrs(
        (
            gp.quicksum([CO2[t, i, c, p] for p in province for i in processes if i != 'BG-CCS' for c in ['AMM', 'Oil']]) -
            gp.quicksum([CO2[t, 'BG-CCS', c, p] for p in province for c in ['AMM', 'Oil']]) +
            gp.quicksum([CO2_METH[t, j, p] for p in province for j in METH_processes[0:6]]) -
            gp.quicksum([CO2_METH[t, 'Biomass-based-CCS', p] for p in province]) -
            CDR_amount[t] +
            gp.quicksum([CO2_cc[t, p] for p in province]) +
            gp.quicksum([
                y_HVC[t, hvc, hvc + '-' + hvc_process, p] *
                HVC_para[hvc][hvc + '-' + hvc_process]['CF'] * 1000
                for hvc in HVCs for hvc_process in HVC_relations[hvc] for p in province
            ]) -
            gp.quicksum([CCS_HVC[t, p] for p in province]) +
            (1 - HVC_para['CCS_end_ratio'][t]) * gp.quicksum([
                y_HVC[t, hvc, hvc + '-' + hvc_process, p] *
                HVC_para[hvc]['Embodied CF'] * 1000
                for hvc in HVCs for hvc_process in HVC_relations[hvc] for p in province
            ]) -
            gp.quicksum([
                y_HVC[t, hp[0:3], hp, p] *
                HVC_para['METH-Embodied CF'] / HVC_para['ETH']['ETH-MTO']['out/in'][t] * 1000
                for hp in ['ETH-MTO', 'PRO-MTO'] for p in province
            ]) +
            gp.quicksum([
                heat_output[t, f, c, p] / Heat_para[f]['heat eff'] / Heat_para[f]['heat value'] *
                Heat_para[f]['CF'] * 1000
                for f in ['Biomass', 'NG'] for c in ['AMM', 'METH'] + HVCs for p in province
            ]) +
            gp.quicksum([
                ele_input[t, c, p] * 1000 * ele_data['CF'][p][t]
                for c in ['AMM', 'METH'] + HVCs for p in province
            ]) -
            gp.quicksum([
                CCS_heat[t, f, c, p]
                for f in ['Biomass', 'NG'] for c in ['AMM', 'METH'] + HVCs for p in province
            ])
            <= SCE_data['Carbon_Target'][t]
            for t in years
        ),
        name='CO2 target'
    )

    # ======================================================================
    # CCS constraints and definitions
    # ======================================================================
    # Maximum HVC CCS (kg)
    model.addConstrs(
        CCS_HVC[t, p] <= 0.9 * gp.quicksum([
            y_HVC[t, hvc, hvc + '-' + hvc_process, p] *
            HVC_para[hvc][hvc + '-' + hvc_process]['CF'] * 1000
            for hvc in HVCs for hvc_process in HVC_relations[hvc]
        ])
        for t in years for p in province
    )

    # Maximum cracking CCS (kg); CO2_cc is net of CCS
    model.addConstrs(
        CCS_cc[t, p] <= 0.9 * (CC_ratio * SCE_data['Demand'][t]['Oil'][p])
        for t in years for p in province
    )

    # Heat CCS upper bounds (kg)
    model.addConstrs(
        CCS_heat[t, 'Biomass', c, p] <=
        heat_output[t, 'Biomass', c, p] / Heat_para['Biomass']['heat eff'] /
        Heat_para['Biomass']['heat value'] * 1000 * (-Heat_para['Biomass']['CF-CCS'])
        for c in ['AMM', 'METH'] + HVCs for t in years for p in province
    )
    model.addConstrs(
        CCS_heat[t, 'NG', c, p] <=
        0.9 * heat_output[t, 'NG', c, p] / Heat_para['NG']['heat eff'] /
        Heat_para['NG']['heat value'] * Heat_para['NG']['CF'] * 1000
        for c in ['AMM', 'METH'] + HVCs for t in years for p in province
    )

    # CCS amount definition (excluding DACCS)
    model.addConstrs(
        CCS_amount[t, p] ==
        gp.quicksum([y[t, i, c, p] * HP[t][i] * CCS_delta_CF[t][i] for i in ['BG-CCS', 'CG-CCS', 'SMR-CCS'] for c in ['AMM', 'Oil']]) +
        gp.quicksum([H2_demand_METH[t, j, p] / H2_demand_ratio['METH_CO_based'] * CCS_delta_CF_METH[t][j] for j in ['Coal-based-CCS', 'Gas-based-CCS', 'Biomass-based-CCS']])
        for t in years for p in province
    )

    model.addConstrs(
        (CO2_cc[t, p] == CC_ratio * SCE_data['Demand'][t]['Oil'][p] - CCS_cc[t, p])
        for t in years for p in province
    )

    # CCS hard cap at 2020
    model.addConstr(gp.quicksum([CCS_amount[2020, p] for p in province]) <= 1000000)

    if SCE_data['CCS'] != 'CCS_limit_off':
        model.addConstrs(
            (
                gp.quicksum([CCS_amount[t, p] for p in province]) +
                gp.quicksum([CCS_cc[t, p] for p in province]) +
                gp.quicksum([CCS_HVC[t, p] for p in province]) +
                gp.quicksum([CCS_heat[t, f, c, p] for f in ['Biomass', 'NG'] for c in ['AMM', 'METH'] + HVCs for p in province])
                <= SCE_data['CCS'] * 1.0E+9
                for t in years
            ),
            name='CCS_limit'
        )

    # ======================================================================
    # CDR constraints (level and growth)
    # ======================================================================
    # CDR maximum level as a fraction of 2020 target emissions
    model.addConstrs(
        (CDR_amount[t] <= SCE_data['CDR_max'] * SCE_data['Carbon_Target'][2020] for t in years),
        name='CDR_level'
    )
    # CDR growth constraint (5-year step with inter)
    model.addConstrs(
        (CDR_amount[t] <= 30000000000 / 5 * inter + 2 * CDR_amount[t - inter] for t in years[1:]),
        name='CDR_level'
    )

    # Do not allow DACCU to decrease (late years constraint)
    model.addConstrs(
        (
            gp.quicksum(H2_demand_METH[t, 'DAC-GH-based', p] for p in province) >=
            gp.quicksum(H2_demand_METH[t - inter, 'DAC-GH-based', p] for p in province)
            for t in years[-2:]
        ),
        name='DAC_cons'
    )

    # ======================================================================
    # Scenario bans / switches (testing)
    # ======================================================================
    # Ban THP and grid-based electrolysis
    model.addConstrs(H2[t, i, c, p] == 0 for t in years for p in province for i in ['Grid-AE', 'Grid-PEM'] for c in chemicals)
    model.addConstrs(X[t, j, p] == 0 for j in ['THP'] for t in years for p in province)

    # Optional hydro/nuclear bans
    if SCE_data['HN_on'] == 'HN_off':
        model.addConstrs(X[t, j, p] == 0 for j in ['Hydro', 'Nuclear'] for t in years for p in province)
    else:
        model.addConstrs(gp.quicksum(U[t, j, p] for p in province) <= 30000000 for j in ['Hydro'] for t in years)
        model.addConstrs(gp.quicksum(U[t, j, p] for p in province) <= 30000000 for j in ['Nuclear'] for t in years)

    # ======================================================================
    # Technology growth-rate sensitivity constraints
    # ======================================================================
    # Note: This is a sensitivity test; the LHS unit must align with SP extraction logic
    # (inputs for GR_upper are expected to be normalized accordingly).
    if GR_upper != 0:
        Tech = list(GR_upper.keys())[0]
        if Tech in ['AE', 'PEM', 'Energy_stroage']:
            model.addConstrs(
                (
                    gp.quicksum(U[t, Tech, p] for p in province) <=
                    (1 + GR_upper[Tech][t]) * gp.quicksum(U[t - inter, Tech, p] for p in province)
                    for t in list(GR_upper[Tech].keys())
                ),
                name=f'{Tech}_cons'
            )
        elif Tech == 'DACCS':
            model.addConstrs(
                (CDR_amount[t] <= (1 + GR_upper[Tech][t]) * CDR_amount[t - inter] for t in list(GR_upper[Tech].keys())),
                name=f'{Tech}_cons'
            )
        elif Tech == 'DACCU':
            model.addConstrs(
                (
                    gp.quicksum(H2_demand_METH[t, 'DAC-GH-based', p] for p in province) <=
                    (1 + GR_upper[Tech][t]) * gp.quicksum(H2_demand_METH[t - inter, 'DAC-GH-based', p] for p in province)
                    for t in list(GR_upper[Tech].keys())
                ),
                name=f'{Tech}_cons'
            )

    model.update()

    # ======================================================================
    # Post-model section (not part of formulation)
    # ======================================================================
    is_lp = not model.isMIP
    if is_lp:
        print("The model is a Linear Programming (LP) model.")
    else:
        print("The model is not a Linear Programming (LP) model (e.g., it's a Mixed-Integer Programming, MIP, model).")

    # Numerical stabilization settings
    model.Params.ScaleFlag = 1
    model.Params.Method = 2              # Barrier method
    model.Params.FeasibilityTol = 0.001

    model.optimize()

    # ======================================================================
    # Feasibility diagnostics
    # ======================================================================
    if model.status == gp.GRB.OPTIMAL:
        print("Optimal solution found.")

    if model.status == gp.GRB.INFEASIBLE:
        model.computeIIS()
        print("Infeasible constraints:")
        for constr in model.getConstrs():
            if constr.IISConstr:
                print(constr.ConstrName)
    else:
        print("Model is feasible or unbounded.")

    # ======================================================================
    # Dual values (shadow prices) extraction
    # ======================================================================
    all_constraints = model.getConstrs()
    shadow_prices = {}
    CDR_prices = {}
    bio_prices = {}
    tech_prices = {}

    target_constraint_name1 = "CO2 target"
    target_constraint_name3 = "CCS_limit"
    bio_prefix = "Biomass_POT"

    for constraint in all_constraints:
        constr_name = constraint.getAttr("ConstrName")

        # CO2 target dual
        if target_constraint_name1 in constr_name:
            shadow_prices[constr_name] = constraint.getAttr("Pi")

        # CCS limit dual (stored in CDR_prices dict as in original code)
        if target_constraint_name3 in constr_name:
            CDR_prices[constr_name] = constraint.getAttr("Pi")

        # Biomass potential dual
        if bio_prefix in constr_name:
            bio_prices[constr_name] = constraint.getAttr("Pi")

        # Tech growth-rate constraint dual
        if GR_upper != 0:
            if Tech + '_cons' in constr_name:
                tech_prices[constr_name] = constraint.getAttr("Pi")

    # ======================================================================
    # Results aggregation initialization
    # ======================================================================
    X_tot = pd.DataFrame(0.0, columns=technologies, index=years)
    x_HVC_tot = pd.DataFrame(0.0, columns=HVC_processes, index=years)
    U_tot = pd.DataFrame(0.0, columns=technologies, index=years)
    u_HVC_tot = pd.DataFrame(0.0, columns=HVC_processes, index=years)
    y_HVC_tot = pd.DataFrame(0.0, columns=HVC_processes, index=years)

    heat_col = (
        ['Biomass-AMM', 'Biomass-METH'] +
        [f'Biomass-{hvc}' for hvc in HVCs] +
        ['NG-AMM', 'NG-METH'] +
        [f'NG-{hvc}' for hvc in HVCs]
    )

    heat_output_tot = pd.DataFrame(0.0, columns=heat_col, index=years)
    ele_input_tot = pd.DataFrame(0.0, columns=['AMM', 'METH'] + HVCs, index=years)

    H2_AMM_tot = pd.DataFrame(0.0, columns=processes, index=years)
    H2_METH_tot = pd.DataFrame(0.0, columns=processes, index=years)
    H2_Oil_tot = pd.DataFrame(0.0, columns=processes, index=years)

    CO2_AMM_tot = pd.DataFrame(0.0, columns=processes, index=years)
    CO2_METH_tot = pd.DataFrame(0.0, columns=METH_processes, index=years)
    CO2_Oil_tot = pd.DataFrame(0.0, columns=processes, index=years)

    CO2_Oil_CC_tot = pd.DataFrame(0.0, columns=['CO2', 'CCS'], index=years)
    CO2_HVC_dir_tot = pd.DataFrame(0.0, columns=HVC_processes + ['CCS_amount'], index=years)
    CO2_HVC_indir_tot = pd.DataFrame(0.0, columns=HVC_processes + ['MTO-re'], index=years)

    CCS_amount_tot = pd.DataFrame(0.0, columns=['CCS_amount'], index=years)

    CO2_heat_tot = pd.DataFrame(0.0, columns=heat_col, index=years)
    CCS_heat_tot = pd.DataFrame(0.0, columns=heat_col, index=years)
    CO2_ele_tot = pd.DataFrame(0.0, columns=['AMM', 'METH'] + HVCs, index=years)

    CO2_tot = pd.DataFrame(
        0.0,
        columns=['AMM', 'Oil', 'METH_burn', 'CDR', 'CC_net', 'HVC_dir_net', 'HVC_indir', 'METH_re', 'heat_net', 'ele'],
        index=years
    )

    cost_AMM_tot = pd.DataFrame(0.0, columns=costs + ['Next'], index=years)
    cost_METH_tot = pd.DataFrame(0.0, columns=costs + ['Next'], index=years)
    cost_next_tot = pd.DataFrame(0.0, columns=[f'{chem}_{co}' for chem in ['AMM', 'METH'] for co in ['CAP', 'OM', 'FS']], index=years)
    cost_Oil_tot = pd.DataFrame(0.0, columns=costs, index=years)

    cost_by_tech_result_tot = pd.DataFrame(0.0, columns=technologies, index=years)
    cost_HVC_tot = pd.DataFrame(0.0, columns=[f'{cs}_{hvc}' for cs in ['CAP', 'OM', 'FS(noMETH)'] for hvc in HVCs] + ['METH_import', 'CCS'], index=years)
    cost_heat_tot = pd.DataFrame(0.0, columns=['AMM', 'METH'] + HVCs + ['CCS'], index=years)
    cost_ele_tot = pd.DataFrame(0.0, columns=['AMM', 'METH'] + HVCs, index=years)

    H2_AMM_group_tot = pd.DataFrame(0.0, columns=H2_groups.keys(), index=years)
    H2_METH_group_tot = pd.DataFrame(0.0, columns=H2_groups.keys(), index=years)
    H2_Oil_group_tot = pd.DataFrame(0.0, columns=H2_groups.keys(), index=years)

    AMM_product_tot = pd.DataFrame(0.0, columns=H2_groups.keys(), index=years)
    METH_product_tot = pd.DataFrame(0.0, columns=METH_processes, index=years)
    Oil_product_tot = pd.DataFrame(0.0, columns=H2_groups.keys(), index=years)

    trans_cost_tot = pd.DataFrame(0.0, columns=chemicals, index=years)

    production_tot = pd.DataFrame(0.0, columns=['AMM', 'METH', 'H2'] + HVCs, index=years)
    LCOX_tot = pd.DataFrame(0.0, columns=['AMM', 'METH', 'H2'] + HVCs, index=years)
    LCOX_tot_with_CC = pd.DataFrame(0.0, columns=['AMM', 'METH', 'H2'] + HVCs, index=years)
    production_value = pd.DataFrame(0.0, columns=['AMM', 'METH', 'H2'] + HVCs, index=years)

    CCS_cost_tot = pd.DataFrame(0.0, columns=['H2_production', 'CC', 'HVC', 'heat'], index=years)

    HVC_FS_input_tot = pd.DataFrame(0.0, columns=HVC_processes, index=years)
    heat_input_tot = pd.DataFrame(0.0, columns=[f'{f}-{c}' for f in ['Biomass', 'NG'] for c in ['AMM', 'METH'] + HVCs], index=years)

    CE_tot = pd.DataFrame(0.0, columns=['AMM', 'METH', 'H2'] + HVCs, index=years)

    # ======================================================================
    # Transportation cost aggregation (national)
    # ======================================================================
    for t in years:
        for c in chemicals:
            trans_cost_tot.at[t, c] = sum(
                trans_couple[t, c, p1, p2].x *
                get_trans_costXdistance(df_distance[df_distance["Province"] == p1][p2].values[0], c, trans_cost)
                for p1 in province for p2 in adjacency_dict[p1]
            )

    CCS_tot = pd.DataFrame(0.0, columns=['H2_production', 'CC', 'HVC', 'heat'], index=years)

    # ======================================================================
    # Shadow price output (undiscounted)
    # ======================================================================
    SP = pd.DataFrame(columns=['Dual Value (without discount)'], index=years)
    CDR_SP = pd.DataFrame(columns=['Dual Value (without discount)'], index=years)

    i = 0
    for constraint_name, shadow_price in shadow_prices.items():
        SP.at[years[i]] = -shadow_price * 1000 / (1 / (1 + R) ** (years[i] - 2020))
        i += 1

    i = 0
    for constraint_name, CDR_price in CDR_prices.items():
        CDR_SP.at[years[i]] = -CDR_price * 1000 / (1 / (1 + R) ** (years[i] - 2020))
        i += 1

    # Biomass shadow prices by province (discount-adjusted; GJ)
    Bio_SP = pd.DataFrame(index=years, columns=province)
    for t in years:
        for p in province:
            constraint_name = f"Biomass_POT[{t},{p}]"
            if constraint_name in bio_prices:
                bio_price = bio_prices[constraint_name]
                Bio_SP.at[t, p] = -bio_price / (1 / (1 + R) ** (t - 2020)) / 1000000000
            else:
                Bio_SP.at[t, p] = np.nan
                print(f"Warning: constraint not found: {constraint_name}")

    # Growth-rate constraint shadow price (if enabled)
    if GR_upper != 0:
        tech_SP = pd.DataFrame(index=years, columns=[Tech])
        i = 0
        for constraint_name, shadow_price in tech_prices.items():
            if Tech in ['AE', 'PEM', 'Energy_stroage']:
                unit = sum(U[list(GR_upper[Tech].keys())[i] - inter, Tech, p].x for p in province)
            elif Tech == 'DACCS':
                unit = CDR_amount[list(GR_upper[Tech].keys())[i] - inter].x
            elif Tech == 'DACCU':
                unit = sum(H2_demand_METH[list(GR_upper[Tech].keys())[i] - inter, 'DAC-GH-based', p].x for p in province)

            tech_SP.at[list(GR_upper[Tech].keys())[i]] = (
                -shadow_price / (1 / (1 + R) ** (list(GR_upper[Tech].keys())[i] - 2020)) *
                unit * GR_low_5_delta[Tech][list(GR_upper[Tech].keys())[i]]
            )
            i += 1

    # ======================================================================
    # Province-level results extraction and Excel export
    # ======================================================================
    for p in province:
        # Tech construction results
        X_results = pd.DataFrame(columns=technologies, index=years)
        for year in years:
            for tech in technologies:
                X_results.at[year, tech] = X[year, tech, p].x
        X_tot += X_results

        x_HVC_results = pd.DataFrame(columns=HVC_processes, index=years)
        y_HVC_results = pd.DataFrame(columns=HVC_processes, index=years)
        u_HVC_results = pd.DataFrame(columns=HVC_processes, index=years)
        HVC_FS_input = pd.DataFrame(columns=[hvc + '-' + hvc_process for hvc in HVCs for hvc_process in HVC_relations[hvc]], index=years)

        for year in years:
            for hvc_pro in HVC_processes:
                x_HVC_results.at[year, hvc_pro] = x_HVC[year, hvc_pro[0:3], hvc_pro, p].x
                y_HVC_results.at[year, hvc_pro] = y_HVC[year, hvc_pro[0:3], hvc_pro, p].x
                u_HVC_results.at[year, hvc_pro] = u_HVC[year, hvc_pro[0:3], hvc_pro, p].x
                HVC_FS_input.at[year, hvc_pro] = y_HVC_results.at[year, hvc_pro] / HVC_para[hvc_pro[0:3]][hvc_pro]['out/in'][year]

        x_HVC_tot += x_HVC_results
        y_HVC_tot += y_HVC_results
        u_HVC_tot += u_HVC_results
        HVC_FS_input_tot += HVC_FS_input

        # ------------------------------------------------------------------
        # Energy-related outputs
        # ------------------------------------------------------------------
        heat_out_GJ = pd.DataFrame(0.0, columns=heat_col, index=years)
        ele_in_MWh = pd.DataFrame(0.0, columns=['AMM', 'METH'] + HVCs, index=years)
        CO2_heat_kg = pd.DataFrame(0.0, columns=heat_col, index=years)
        CCS_heat_kg = pd.DataFrame(0.0, columns=heat_col, index=years)
        CO2_ele_kg = pd.DataFrame(0.0, columns=['AMM', 'METH'] + HVCs, index=years)
        cost_heat = pd.DataFrame(0.0, columns=['AMM', 'METH'] + HVCs + ['CCS'], index=years)
        cost_ele = pd.DataFrame(0.0, columns=['AMM', 'METH'] + HVCs, index=years)
        heat_input = pd.DataFrame(columns=heat_col, index=years)

        for t in years:
            for c in ['AMM', 'METH'] + HVCs:
                ele_in_MWh.at[t, c] = ele_input[t, c, p].x
                CO2_ele_kg.at[t, c] = ele_input[t, c, p].x * 1000 * ele_data['CF'][p][t]
                cost_heat.at[t, c] = sum(
                    heat_output[t, f, c, p].x *
                    (Heat_para[f]['CAP+OM'] + 1 / Heat_para[f]['heat eff'] / Heat_para[f]['heat value'] * Heat_para[f]['FS'][t])
                    for f in ['Biomass', 'NG']
                )
                cost_ele.at[t, c] = ele_input[t, c, p].x * 1000 * ele_data['cost'][p][t]

                for f in ['Biomass', 'NG']:
                    heat_out_GJ.at[t, f'{f}-{c}'] = heat_output[t, f, c, p].x
                    heat_input.at[t, f'{f}-{c}'] = heat_out_GJ.at[t, f'{f}-{c}'] / Heat_para[f]['heat eff']
                    CO2_heat_kg.at[t, f'{f}-{c}'] = (
                        heat_output[t, f, c, p].x / Heat_para[f]['heat eff'] /
                        Heat_para[f]['heat value'] * Heat_para[f]['CF'] * 1000
                    )
                    CCS_heat_kg.at[t, f'{f}-{c}'] = CCS_heat[t, f, c, p].x

            cost_heat.at[t, 'CCS'] = sum(CCS_heat[t, f, c, p].x for f in ['Biomass', 'NG'] for c in ['AMM', 'METH'] + HVCs)

        heat_output_tot += heat_out_GJ
        heat_input_tot += heat_input
        ele_input_tot += ele_in_MWh
        CO2_heat_tot += CO2_heat_kg
        CCS_heat_tot += CCS_heat_kg
        CO2_ele_tot += CO2_ele_kg
        cost_heat_tot += cost_heat
        cost_ele_tot += cost_ele

        # ------------------------------------------------------------------
        # Technology capacity results
        # ------------------------------------------------------------------
        U_results = pd.DataFrame(columns=technologies, index=years)
        for year in years:
            for tech in technologies:
                U_results.at[year, tech] = U[year, tech, p].x
        U_tot += U_results

        # ------------------------------------------------------------------
        # Hydrogen input results (back-calculated to operation scale)
        # ------------------------------------------------------------------
        H2_AMM = pd.DataFrame(columns=processes, index=years)
        for year in years:
            for process in processes:
                H2_AMM.at[year, process] = H2[year, process, 'AMM', p].x / HP[year][process]
        H2_AMM_tot += H2_AMM

        H2_METH = pd.DataFrame(columns=processes, index=years)
        for year in years:
            for process in processes:
                H2_METH.at[year, process] = H2[year, process, 'METH', p].x / HP[year][process]
        H2_METH_tot += H2_METH

        H2_Oil = pd.DataFrame(columns=processes, index=years)
        for year in years:
            for process in processes:
                H2_Oil.at[year, process] = H2[year, process, 'Oil', p].x / HP[year][process]
        H2_Oil_tot += H2_Oil

        # ------------------------------------------------------------------
        # CO2 results (sign adjustments for biomass CCS etc.)
        # ------------------------------------------------------------------
        CO2_AMM = pd.DataFrame(columns=processes, index=years)
        for year in years:
            for process in processes:
                CO2_AMM.at[year, process] = -CO2[year, process, 'AMM', p].x if process == 'BG-CCS' else CO2[year, process, 'AMM', p].x
        CO2_AMM_tot += CO2_AMM

        CO2_METH_data = pd.DataFrame(columns=METH_processes, index=years)
        for year in years:
            for process in METH_processes:
                if process in ['Biomass-based', 'Biomass-based-CCS', 'DAC-GH-based']:
                    CO2_METH_data.at[year, process] = -CO2_METH[year, process, p].x
                else:
                    CO2_METH_data.at[year, process] = CO2_METH[year, process, p].x
        CO2_METH_tot += CO2_METH_data

        CO2_Oil = pd.DataFrame(columns=processes, index=years)
        for year in years:
            for process in processes:
                CO2_Oil.at[year, process] = -CO2[year, process, 'Oil', p].x if process == 'BG-CCS' else CO2[year, process, 'Oil', p].x
        CO2_Oil_tot += CO2_Oil

        CO2_Oil_CC = pd.DataFrame(columns=['CO2', 'CCS'], index=years)
        for year in years:
            CO2_Oil_CC.at[year, 'CO2'] = CO2_cc[year, p].x
            CO2_Oil_CC.at[year, 'CCS'] = CCS_cc[year, p].x
        CO2_Oil_CC_tot += CO2_Oil_CC

        # ------------------------------------------------------------------
        # HVC CO2 (direct and embodied)
        # ------------------------------------------------------------------
        CO2_HVC_dir = pd.DataFrame(0.0, columns=HVC_processes + ['CCS_amount'], index=years)
        CO2_HVC_indir = pd.DataFrame(0.0, columns=HVC_processes + ['MTO-re'], index=years)

        for year in years:  # all in kg CO2
            for HVC_pro in HVC_processes:
                CO2_HVC_dir.at[year, HVC_pro] = y_HVC[year, HVC_pro[0:3], HVC_pro, p].x * HVC_para[HVC_pro[0:3]][HVC_pro]['CF'] * 1000
                CO2_HVC_indir.at[year, HVC_pro] = (1 - HVC_para['CCS_end_ratio'][year]) * y_HVC[year, HVC_pro[0:3], HVC_pro, p].x * HVC_para[HVC_pro[0:3]]['Embodied CF'] * 1000

            CO2_HVC_dir.at[year, 'CCS_amount'] = -CCS_HVC[year, p].x
            CO2_HVC_indir.at[year, 'MTO-re'] = -(
                (y_HVC[year, 'ETH', 'ETH-MTO', p].x + y_HVC[year, 'PRO', 'PRO-MTO', p].x) *
                HVC_para['METH-Embodied CF'] / HVC_para['ETH']['ETH-MTO']['out/in'][t] * 1000
            )

        CO2_HVC_dir_tot += CO2_HVC_dir
        CO2_HVC_indir_tot += CO2_HVC_indir

        # ------------------------------------------------------------------
        # CCS amounts (province)
        # ------------------------------------------------------------------
        CCS_amount_results = pd.DataFrame(columns=['CCS_amount'], index=years)
        for year in years:
            CCS_amount_results.at[year, 'CCS_amount'] = 0 if year == 2020 else CCS_amount[year, p].x
        CCS_amount_tot += CCS_amount_results

        # ------------------------------------------------------------------
        # Cost outputs (AMM, METH, Oil, tech)
        # ------------------------------------------------------------------
        cost_by_year_results_AMM = pd.DataFrame(columns=costs + ['Next'], index=years)
        for year in years:
            for cost in costs:
                cost_by_year_results_AMM.at[year, cost] = cost_by_year[year, cost, 'AMM', p].x
            cost_by_year_results_AMM.at[year, 'Next'] = next_cost_by_year[year, 'AMM', p].x
        cost_AMM_tot += cost_by_year_results_AMM

        cost_by_year_results_METH = pd.DataFrame(columns=costs + ['Next'], index=years)
        for year in years:
            for cost in costs:
                cost_by_year_results_METH.at[year, cost] = cost_by_year[year, cost, 'METH', p].x
            cost_by_year_results_METH.at[year, 'Next'] = next_cost_by_year[year, 'METH', p].x
        cost_METH_tot += cost_by_year_results_METH

        cost_by_tech_result = pd.DataFrame(columns=technologies, index=years)
        for t in years:
            for j in technologies:
                cost_by_tech_result.at[t, j] = cost_by_tech[t, j, p].x
        cost_by_tech_result_tot += cost_by_tech_result

        # Next-cost decomposition (CAP/OM/FS) for AMM and METH
        cost_next = pd.DataFrame(columns=[f'{chem}_{co}' for chem in ['AMM', 'METH'] for co in ['CAP', 'OM', 'FS']], index=years)
        for t in years:
            for co in ['CAP', 'OM', 'FS']:
                cost_next.at[t, f'AMM_{co}'] = (
                    sum(y[t, i, 'AMM', p].x * HP[t][i] for i in processes) /
                    H2_demand_ratio['AMM'] * NC[t]['AMM'][co]
                )
            for co in ['CAP', 'OM', 'FS']:
                cost_next.at[t, f'METH_{co}'] = (
                    sum(H2_demand_METH[t, k, p].x for k in METH_processes[:-2]) /
                    H2_demand_ratio['METH_CO_based'] * NC[t]['METH1'][co] +
                    H2_demand_METH[t, METH_processes[-2], p].x /
                    H2_demand_ratio['METH_CO2_based'] * NC[t]['METH2'][co] +
                    H2_demand_METH[t, METH_processes[-1], p].x /
                    H2_demand_ratio['METH_CO2_based'] * NC[t]['METH3'][co]
                )
        cost_next_tot += cost_next

        cost_by_year_results_Oil = pd.DataFrame(columns=costs, index=years)
        for year in years:
            for cost in costs:
                cost_by_year_results_Oil.at[year, cost] = cost_by_year[year, cost, 'Oil', p].x
        cost_Oil_tot += cost_by_year_results_Oil

        # HVC cost breakdown
        cost_HVC = pd.DataFrame(columns=[f'{cs}_{hvc}' for cs in ['CAP', 'OM', 'FS(noMETH)'] for hvc in HVCs] + ['METH_import', 'CCS'], index=years)
        for year in years:
            for hvc in HVCs:
                for cs in ['CAP', 'OM']:
                    cost_HVC.at[year, f'{cs}_{hvc}'] = sum(
                        HVC_para[hvc][hvc + '-' + hvc_process][cs] *
                        (
                            1000000 * EXCAP[year][hvc + '-' + hvc_process] * EXP_chemical_ratio[hvc + '-' + hvc_process][p] +
                            sum(x_HVC[tau, hvc, hvc + '-' + hvc_process, p].x for tau in years if tau <= year and tau + LT[tau][hvc + '-' + hvc_process] > year)
                        )
                        for hvc_process in HVC_relations[hvc]
                    )
                cost_HVC.at[year, f'FS(noMETH)_{hvc}'] = sum(
                    HVC_para[hvc][hvc + '-' + hvc_process]['FS-cost'][year] /
                    HVC_para[hvc][hvc + '-' + hvc_process]['out/in'][t] *
                    y_HVC[year, hvc, hvc + '-' + hvc_process, p].x
                    for hvc_process in HVC_relations[hvc] if hvc_process != 'MTO'
                )

            cost_HVC.at[year, 'METH_import'] = (
                HVC_para['ETH']['ETH-MTO']['FS-cost'][year] *
                demand_all['METH']['MTO_import'][year] *
                DS_ratio['Demand_ratio'][p]['METH_Import_Share'] * 1000000
            )
            cost_HVC.at[year, 'CCS'] = CCS_cost[p][year] * 0.001 * (CCS_HVC[year, p].x)

        cost_HVC_tot += cost_HVC

        # ======================================================================
        # Hydrogen supply grouping (AMM / METH / Oil)
        # ======================================================================
        H2_groupby_AMM = pd.DataFrame(columns=H2_groups.keys(), index=years)
        for t in years:
            for j in H2_groups.keys():
                H2_groupby_AMM.at[t, j] = sum(H2[t, i, 'AMM', p].x for i in H2_groups[j])
        H2_AMM_group_tot += H2_groupby_AMM

        H2_groupby_METH = pd.DataFrame(columns=H2_groups.keys(), index=years)
        for t in years:
            for j in H2_groups.keys():
                H2_groupby_METH.at[t, j] = sum(H2[t, i, 'METH', p].x for i in H2_groups[j])
        H2_METH_group_tot += H2_groupby_METH

        H2_groupby_Oil = pd.DataFrame(columns=H2_groups.keys(), index=years)
        for t in years:
            for j in H2_groups.keys():
                H2_groupby_Oil.at[t, j] = sum(H2[t, i, 'Oil', p].x for i in H2_groups[j])
        H2_Oil_group_tot += H2_groupby_Oil

        # ======================================================================
        # Product outputs (convert H2 to product quantities)
        # ======================================================================
        AMM_product = pd.DataFrame(columns=H2_groups.keys(), index=years)
        for t in years:
            for j in H2_groups.keys():
                AMM_product.at[t, j] = sum(H2[t, i, 'AMM', p].x for i in H2_groups[j]) / 0.18
        AMM_product_tot += AMM_product

        METH_product = pd.DataFrame(columns=METH_processes, index=years)
        for t in years:
            for j in METH_processes:
                if j == 'CO2-GH-based' or j == 'DAC-GH-based':
                    METH_product.at[t, j] = H2_demand_METH[t, j, p].x / 0.18
                else:
                    METH_product.at[t, j] = H2_demand_METH[t, j, p].x / 0.128
        METH_product_tot += METH_product

        Oil_product = pd.DataFrame(columns=H2_groups.keys(), index=years)
        for t in years:
            for j in H2_groups.keys():
                Oil_product.at[t, j] = sum(H2[t, i, 'Oil', p].x for i in H2_groups[j]) / 0.012
        Oil_product_tot += Oil_product

        # ======================================================================
        # Total production by product (tonnes, and HVC in tonnes)
        # ======================================================================
        production_p = pd.DataFrame(columns=['AMM', 'METH', 'H2'] + HVCs, index=years)
        for t in years:
            production_p.at[t, 'AMM'] = (
                sum(AMM_product.at[t, j] for j in [
                    'Wind', 'Solar', 'Hydro', 'Nuclear', 'Grid', 'BG', 'CG', 'SMR', 'EXT',
                    'BG-CCS', 'CG-CCS', 'SMR-CCS'
                ]) / 1000
            )  # tonnes
            production_p.at[t, 'METH'] = sum(METH_product.at[t, j] for j in METH_processes) / 1000  # tonnes
            production_p.at[t, 'H2'] = (
                sum(H2_groupby_Oil.at[t, j] for j in [
                    'Wind', 'Solar', 'Hydro', 'Nuclear', 'Grid', 'BG', 'CG', 'SMR', 'EXT',
                    'BG-CCS', 'CG-CCS', 'SMR-CCS'
                ]) / 1000
            )  # tonnes
            for hvc in HVCs:
                production_p.at[t, hvc] = sum(y_HVC_results.at[t, f'{hvc}-{pro}'] for pro in HVC_relations[hvc])
        production_tot += production_p

        # ======================================================================
        # Transport matrices (example year 2060)
        # ======================================================================
        trans_AMM = pd.DataFrame(columns=province, index=province)
        trans_METH = pd.DataFrame(columns=province, index=province)
        trans_H2 = pd.DataFrame(columns=province, index=province)
        for p1 in province:
            for p2 in adjacency_dict[p1]:
                if p2 != 'Tibet':
                    trans_AMM.at[p1, p2] = trans_couple[2060, 'AMM', p1, p2].x
                    trans_METH.at[p1, p2] = trans_couple[2060, 'METH', p1, p2].x
                    trans_H2.at[p1, p2] = trans_couple[2060, 'Oil', p1, p2].x

        # ======================================================================
        # Aggregate CO2 components
        # ======================================================================
        CO2_sum = pd.DataFrame(
            0.0,
            columns=['AMM', 'Oil', 'METH_burn', 'CDR', 'CC_net', 'HVC_dir_net', 'HVC_indir', 'METH_re', 'heat_net', 'ele'],
            index=years
        )
        CO2_sum['AMM'] = CO2_AMM.sum(axis=1)
        CO2_sum['Oil'] = CO2_Oil.sum(axis=1)
        CO2_sum['METH_burn'] = CO2_METH_data.sum(axis=1)
        CO2_sum['CC_net'] = CO2_Oil_CC['CO2']
        CO2_sum['HVC_dir_net'] = CO2_HVC_dir.sum(axis=1)
        CO2_sum['HVC_indir'] = CO2_HVC_indir.drop(columns=['MTO-re'], errors='ignore').sum(axis=1)
        CO2_sum['METH_re'] = CO2_HVC_indir['MTO-re']
        CO2_sum['heat_net'] = CO2_heat_kg.sum(axis=1) - CCS_heat_kg.sum(axis=1)
        CO2_sum['ele'] = CO2_ele_kg.sum(axis=1)

        # Allocate pre-2060 CDR using 2060 proportional shares (legacy behavior)
        for t in years:
            CO2_sum.at[t, 'CDR'] = (
                -CDR_amount[t].x *
                (sum(CO2_sum.at[2060, c] for c in ['AMM', 'Oil', 'METH_burn', 'CC_net', 'HVC_dir_net', 'HVC_indir', 'METH_re', 'heat_net', 'ele'])) /
                (CDR_amount[2060].x + 0.000001)
            )
        CO2_tot += CO2_sum

        # ======================================================================
        # CDR amount and cost (province bookkeeping; national CDR overwritten later)
        # ======================================================================
        CDR_amount_results = pd.DataFrame(columns=['CDR_amount'], index=years)
        for t in years:
            CDR_amount_results.at[t, 'CDR_amount'] = -CO2_sum.at[t, 'CDR']  # positive
            CO2_tot.at[t, 'CDR'] = -CDR_amount[t].x  # overwrite with national-level CDR from model variable

        CDR_cost_results = pd.DataFrame(columns=['CDR_cost'], index=years)
        for t in years:
            CDR_cost_results.at[t, 'CDR_cost'] = -CO2_sum.at[t, 'CDR'] * SCE_data['CDR_cost'][t] * 0.001  # positive

        # ======================================================================
        # LCOX calculation (excluding CDR; pure production cost)
        # ======================================================================
        LCOX_p = pd.DataFrame(columns=['AMM', 'METH', 'H2'] + HVCs, index=years)
        for t in years:
            LCOX_p.at[t, 'AMM'] = (
                sum(cost_by_year_results_AMM.at[t, c] for c in ['CAP', 'OM', 'FS', 'CCS', 'Next']) +
                cost_ele.at[t, 'AMM'] -
                cost_ele.at[2020, 'AMM'] * production_p.at[t, 'AMM'] / (production_p.at[2020, 'AMM'] + 0.000001) +
                cost_heat.at[t, 'AMM'] -
                cost_heat.at[2020, 'AMM'] * production_p.at[t, 'AMM'] / (production_p.at[2020, 'AMM'] + 0.000001) +
                cost_heat.at[t, 'CCS'] *
                sum(CCS_heat_kg.at[t, f'{fs}-{hvc}'] for fs in ['Biomass', 'NG'] for hvc in ['AMM']) /
                sum(0.000001 + CCS_heat_kg.at[t, f'{fs}-{hvc}'] for fs in ['Biomass', 'NG'] for hvc in ['AMM', 'METH'] + HVCs)
            ) / (production_p.at[t, 'AMM'] + 0.0000001)  # RMB/tonne

            LCOX_p.at[t, 'METH'] = (
                sum(cost_by_year_results_METH.at[t, c] for c in ['CAP', 'OM', 'FS', 'CCS', 'Next']) +
                cost_ele.at[t, 'METH'] -
                cost_ele.at[2020, 'METH'] * production_p.at[t, 'METH'] / (production_p.at[2020, 'METH'] + 0.000001) +
                cost_heat.at[t, 'METH'] -
                cost_heat.at[2020, 'METH'] * production_p.at[t, 'METH'] / (production_p.at[2020, 'METH'] + 0.000001) +
                cost_heat.at[t, 'CCS'] *
                sum(CCS_heat_kg.at[t, f'{fs}-{hvc}'] for fs in ['Biomass', 'NG'] for hvc in ['METH']) /
                sum(0.000001 + CCS_heat_kg.at[t, f'{fs}-{hvc}'] for fs in ['Biomass', 'NG'] for hvc in ['AMM', 'METH'] + HVCs)
            ) / (production_p.at[t, 'METH'] + 0.000001)  # RMB/tonne

            LCOX_p.at[t, 'H2'] = (
                sum(cost_by_year_results_Oil.at[t, c] for c in ['CAP', 'OM', 'FS', 'CCS'])
            ) / (production_p.at[t, 'H2'] + 0.00001)  # RMB/tonne

            # ETH / PRO / BTX: allocate methanol-to-olefins cost share and CCS by direct emissions
            LCOX_p.at[t, 'ETH'] = (
                sum(cost_HVC.at[t, c] for c in ['CAP_ETH', 'OM_ETH', 'FS(noMETH)_ETH']) +
                LCOX_p.at[t, 'METH'] * y_HVC_results.at[t, 'ETH-MTO'] / HVC_para['ETH']['ETH-MTO']['out/in'][t] * 0.8 +
                cost_HVC.at[t, 'CCS'] *
                (sum(CO2_HVC_dir.at[t, f'{"ETH"}-{hvc_pro}'] for hvc_pro in HVC_relations['ETH']) /
                 (sum(CO2_HVC_dir.at[t, f'{hvc}-{hvc_pro}'] for hvc in HVCs for hvc_pro in HVC_relations[hvc]) + 0.00001)) +
                cost_ele.at[t, 'ETH'] -
                cost_ele.at[2020, 'ETH'] * production_p.at[t, 'ETH'] / (production_p.at[2020, 'ETH'] + 0.000001) +
                cost_heat.at[t, 'ETH'] -
                cost_heat.at[2020, 'ETH'] * production_p.at[t, 'ETH'] / (production_p.at[2020, 'ETH'] + 0.000001) +
                cost_heat.at[t, 'CCS'] *
                sum(CCS_heat_kg.at[t, f'{fs}-{hvc}'] for fs in ['Biomass', 'NG'] for hvc in ['ETH']) /
                sum(0.000001 + CCS_heat_kg.at[t, f'{fs}-{hvc}'] for fs in ['Biomass', 'NG'] for hvc in ['AMM', 'METH'] + HVCs)
            ) / (production_p.at[t, 'ETH'] + 0.00001)

            LCOX_p.at[t, 'PRO'] = (
                sum(cost_HVC.at[t, c] for c in ['CAP_PRO', 'OM_PRO', 'FS(noMETH)_PRO']) +
                LCOX_p.at[t, 'METH'] * y_HVC_results.at[t, 'PRO-MTO'] / HVC_para['PRO']['PRO-MTO']['out/in'][t] * 0.8 +
                cost_HVC.at[t, 'CCS'] *
                (sum(CO2_HVC_dir.at[t, f'{"PRO"}-{hvc_pro}'] for hvc_pro in HVC_relations['PRO']) /
                 (sum(CO2_HVC_dir.at[t, f'{hvc}-{hvc_pro}'] for hvc in HVCs for hvc_pro in HVC_relations[hvc]) + 0.00001)) +
                cost_ele.at[t, 'PRO'] -
                cost_ele.at[2020, 'PRO'] * production_p.at[t, 'PRO'] / (production_p.at[2020, 'PRO'] + 0.000001) +
                cost_heat.at[t, 'PRO'] -
                cost_heat.at[2020, 'PRO'] * production_p.at[t, 'PRO'] / (production_p.at[2020, 'PRO'] + 0.000001) +
                cost_heat.at[t, 'CCS'] *
                sum(CCS_heat_kg.at[t, f'{fs}-{hvc}'] for fs in ['Biomass', 'NG'] for hvc in ['PRO']) /
                sum(0.000001 + CCS_heat_kg.at[t, f'{fs}-{hvc}'] for fs in ['Biomass', 'NG'] for hvc in ['AMM', 'METH'] + HVCs)
            ) / (production_p.at[t, 'PRO'] + 0.00001)

            LCOX_p.at[t, 'BTX'] = (
                sum(cost_HVC.at[t, c] for c in ['CAP_BTX', 'OM_BTX', 'FS(noMETH)_BTX']) +
                cost_HVC.at[t, 'CCS'] *
                (sum(CO2_HVC_dir.at[t, f'{"BTX"}-{hvc_pro}'] for hvc_pro in HVC_relations['BTX']) /
                 (sum(CO2_HVC_dir.at[t, f'{hvc}-{hvc_pro}'] for hvc in HVCs for hvc_pro in HVC_relations[hvc]) + 0.00001)) +
                cost_ele.at[t, 'BTX'] -
                cost_ele.at[2020, 'BTX'] * production_p.at[t, 'BTX'] / (production_p.at[2020, 'BTX'] + 0.000001) +
                cost_heat.at[t, 'BTX'] -
                cost_heat.at[2020, 'BTX'] * production_p.at[t, 'BTX'] / (production_p.at[2020, 'BTX'] + 0.000001) +
                cost_heat.at[t, 'CCS'] *
                sum(CCS_heat_kg.at[t, f'{fs}-{hvc}'] for fs in ['Biomass', 'NG'] for hvc in ['BTX']) /
                sum(0.000001 + CCS_heat_kg.at[t, f'{fs}-{hvc}'] for fs in ['Biomass', 'NG'] for hvc in ['AMM', 'METH'] + HVCs)
            ) / (production_p.at[t, 'BTX'] + 0.00001)

        # ======================================================================
        # Carbon emissions per product (kg CO2)
        # ======================================================================
        CE_p = pd.DataFrame(columns=['AMM', 'METH', 'H2'] + HVCs, index=years)
        for t in years:
            CE_p.at[t, 'AMM'] = (
                CO2_sum.at[t, 'AMM'] + CO2_ele_kg.at[t, 'AMM'] +
                sum(CO2_heat_kg.at[t, f'{fs}-{c}'] - CCS_heat_kg.at[t, f'{fs}-{c}'] for fs in ['Biomass', 'NG'] for c in ['AMM'])
            )
            CE_p.at[t, 'METH'] = (
                CO2_sum.at[t, 'METH_burn'] + CO2_ele_kg.at[t, 'METH'] +
                sum(CO2_heat_kg.at[t, f'{fs}-{c}'] - CCS_heat_kg.at[t, f'{fs}-{c}'] for fs in ['Biomass', 'NG'] for c in ['METH']) +
                CO2_sum.at[t, 'METH_re']
            )
            CE_p.at[t, 'H2'] = CO2_sum.at[t, 'Oil']

            CE_p.at[t, 'ETH'] = (
                sum(CO2_HVC_dir.at[t, f'ETH-{pro}'] for pro in HVC_relations['ETH']) +
                sum(CO2_HVC_indir.at[t, f'ETH-{pro}'] for pro in HVC_relations['ETH']) +
                CO2_HVC_dir.at[t, 'CCS_amount'] *
                sum(CO2_HVC_dir.at[t, f'ETH-{pro}'] for pro in HVC_relations['ETH']) /
                sum(0.00001 + CO2_HVC_dir.at[t, f'{hvc}-{pro}'] for hvc in HVCs for pro in HVC_relations[hvc]) +
                CO2_ele_kg.at[t, 'ETH'] +
                sum(CO2_heat_kg.at[t, f'{fs}-{c}'] - CCS_heat_kg.at[t, f'{fs}-{c}'] for fs in ['Biomass', 'NG'] for c in ['ETH'])
            )
            CE_p.at[t, 'PRO'] = (
                sum(CO2_HVC_dir.at[t, f'PRO-{pro}'] for pro in HVC_relations['PRO']) +
                sum(CO2_HVC_indir.at[t, f'PRO-{pro}'] for pro in HVC_relations['PRO']) +
                CO2_HVC_dir.at[t, 'CCS_amount'] *
                sum(CO2_HVC_dir.at[t, f'PRO-{pro}'] for pro in HVC_relations['PRO']) /
                sum(0.00001 + CO2_HVC_dir.at[t, f'{hvc}-{pro}'] for hvc in HVCs for pro in HVC_relations[hvc]) +
                CO2_ele_kg.at[t, 'PRO'] +
                sum(CO2_heat_kg.at[t, f'{fs}-{c}'] - CCS_heat_kg.at[t, f'{fs}-{c}'] for fs in ['Biomass', 'NG'] for c in ['PRO'])
            )
            CE_p.at[t, 'BTX'] = (
                sum(CO2_HVC_dir.at[t, f'BTX-{pro}'] for pro in HVC_relations['BTX']) +
                sum(CO2_HVC_indir.at[t, f'BTX-{pro}'] for pro in HVC_relations['BTX']) +
                CO2_HVC_dir.at[t, 'CCS_amount'] *
                sum(CO2_HVC_dir.at[t, f'BTX-{pro}'] for pro in HVC_relations['BTX']) /
                sum(0.00001 + CO2_HVC_dir.at[t, f'{hvc}-{pro}'] for hvc in HVCs for pro in HVC_relations[hvc]) +
                CO2_ele_kg.at[t, 'BTX'] +
                sum(CO2_heat_kg.at[t, f'{fs}-{c}'] - CCS_heat_kg.at[t, f'{fs}-{c}'] for fs in ['Biomass', 'NG'] for c in ['BTX'])
            )

        CE_tot += CE_p

        # ======================================================================
        # LCOX with carbon cost (using CO2 shadow price)
        # ======================================================================
        LCOX_with_CC = pd.DataFrame(columns=['AMM', 'METH', 'H2'] + HVCs, index=years)
        for t in years:
            for c in ['AMM', 'METH', 'H2'] + HVCs:
                LCOX_with_CC.at[t, c] = (
                    LCOX_p.at[t, c] +
                    CE_p.at[t, c] / (production_p.at[t, c] + 0.000001) *
                    SP.at[t, 'Dual Value (without discount)'] / 1000
                )  # kg -> tonne

        # ======================================================================
        # Production value (cost-based, no profit/tax/carbon tax)
        # ======================================================================
        production_value_p = pd.DataFrame(columns=['AMM', 'METH', 'H2'] + HVCs, index=years)
        for t in years:
            for c in ['AMM', 'METH', 'H2'] + HVCs:
                production_value_p.at[t, c] = production_p.at[t, c] * LCOX_p.at[t, c]
        production_value += production_value_p

        # ======================================================================
        # CCS totals and CCS cost by segment (province)
        # ======================================================================
        CCS_p = pd.DataFrame(0.0, columns=['H2_production', 'CC', 'HVC', 'heat'], index=years)
        CCS_cost_p = pd.DataFrame(0.0, columns=['H2_production', 'CC', 'HVC', 'heat'], index=years)
        for t in years:
            if t == 2020:
                CCS_p.at[t, 'H2_production'] = 0
            else:
                CCS_p.at[t, 'H2_production'] = CCS_amount[t, p].x

            CCS_p.at[t, 'CC'] = CCS_cc[t, p].x
            CCS_p.at[t, 'HVC'] = CCS_HVC[t, p].x
            CCS_p.at[t, 'heat'] = sum(CCS_heat[t, f, c, p].x for f in ['Biomass', 'NG'] for c in ['AMM', 'METH'] + HVCs)

            for col in ['H2_production', 'CC', 'HVC', 'heat']:
                CCS_cost_p.at[t, col] = CCS_p.at[t, col] * CCS_cost[p][t] * 0.001

        CCS_tot += CCS_p
        CCS_cost_tot += CCS_cost_p

        # ======================================================================
        # Province-level Excel export
        # ======================================================================
        with pd.ExcelWriter('{}_{}.xlsx'.format(store_name, p)) as writer:
            X_results.to_excel(writer, sheet_name='X-tech-construction')
            x_HVC_results.to_excel(writer, sheet_name='x_HVC_construction')
            U_results.to_excel(writer, sheet_name='u-tech-capacity')
            u_HVC_results.to_excel(writer, sheet_name='u_HVC_capacity')
            y_HVC_results.to_excel(writer, sheet_name='y_HVC_production')
            production_p.to_excel(writer, sheet_name='production_t')
            H2_AMM.to_excel(writer, sheet_name='input-KWh-AMM')
            H2_METH.to_excel(writer, sheet_name='input-KWh-METH')
            H2_Oil.to_excel(writer, sheet_name='input-KWh-Oil')
            HVC_FS_input.to_excel(writer, sheet_name='HVC_feedstock_t')
            CO2_AMM.to_excel(writer, sheet_name='CO2-emission-AMM')
            CO2_METH_data.to_excel(writer, sheet_name='CO2-emission-METH')
            CO2_Oil.to_excel(writer, sheet_name='CO2-emission-Oil')
            CO2_Oil_CC.to_excel(writer, sheet_name='CO2_Oil_CC')
            CO2_HVC_dir.to_excel(writer, sheet_name='CO2_HVC_dir')
            CO2_HVC_indir.to_excel(writer, sheet_name='CO2_HVC_indir')
            CCS_amount_results.to_excel(writer, sheet_name='CCS_amount')
            CE_p.to_excel(writer, sheet_name='CO2_by_product')
            cost_by_tech_result.to_excel(writer, sheet_name='CAPEX')
            cost_by_year_results_AMM.to_excel(writer, sheet_name='cost_AMM')
            cost_by_year_results_METH.to_excel(writer, sheet_name='cost_METH')
            cost_next.to_excel(writer, sheet_name='cost_next')
            cost_by_year_results_Oil.to_excel(writer, sheet_name='cost_Oil')
            cost_HVC.to_excel(writer, sheet_name='cost_HVC')
            LCOX_p.to_excel(writer, sheet_name='LCOX_rmb_per_t')
            LCOX_with_CC.to_excel(writer, sheet_name='LCOX_wCC_rmb_per_t')
            production_value_p.to_excel(writer, sheet_name='production_value')
            H2_groupby_AMM.to_excel(writer, sheet_name='H2_AMM')
            H2_groupby_METH.to_excel(writer, sheet_name='H2_METH')
            H2_groupby_Oil.to_excel(writer, sheet_name='H2_Oil')
            AMM_product.to_excel(writer, sheet_name='AMM_product')
            METH_product.to_excel(writer, sheet_name='METH_product')
            Oil_product.to_excel(writer, sheet_name='Oil_product')
            CDR_amount_results.to_excel(writer, sheet_name='CDR_amount')
            CDR_cost_results.to_excel(writer, sheet_name='CDR_cost')
            heat_out_GJ.to_excel(writer, sheet_name='heat_output_GJ')
            heat_input.to_excel(writer, sheet_name='heat_input_GJ')
            ele_in_MWh.to_excel(writer, sheet_name='ele_input_MWh')
            CO2_heat_kg.to_excel(writer, sheet_name='CO2_heat_kg')
            CO2_ele_kg.to_excel(writer, sheet_name='CO2_ele_kg')
            CO2_sum.to_excel(writer, sheet_name='CO2_sum_kg')
            cost_heat.to_excel(writer, sheet_name='cost_heat')
            cost_ele.to_excel(writer, sheet_name='cost_ele')
            CCS_heat_kg.to_excel(writer, sheet_name='CCS_heat')
            CCS_p.to_excel(writer, sheet_name='CCS_tot_kg')
            CCS_cost_p.to_excel(writer, sheet_name='CCS_cost_sum')
            SP.to_excel(writer, sheet_name='shadow prices_t')

    # ======================================================================
    # National CDR and CDR cost (model-level totals)
    # ======================================================================
    CDR_amount_tot = pd.DataFrame(0.0, columns=['CDR_amount'], index=years)
    for t in years:
        CDR_amount_tot.at[t, 'CDR_amount'] = CDR_amount[t].x

    CDR_cost_tot = pd.DataFrame(0.0, columns=['CDR_cost'], index=years)
    for t in years:
        CDR_cost_tot.at[t, 'CDR_cost'] = CDR_amount[t].x * SCE_data['CDR_cost'][t] * 0.001

    # ======================================================================
    # National weighted-average LCOX
    # ======================================================================
    for t in years:
        for c in ['AMM', 'METH', 'H2'] + HVCs:
            LCOX_tot.at[t, c] = production_value.at[t, c] / (production_tot.at[t, c] + 0.00001)
            LCOX_tot_with_CC.at[t, c] = (
                LCOX_tot.at[t, c] +
                CE_tot.at[t, c] / (production_p.at[t, c] + 0.000001) *
                SP.at[t, 'Dual Value (without discount)'] / 1000
            )

    # ======================================================================
    # Write national aggregated results to Excel
    # ======================================================================
    with pd.ExcelWriter(f'{store_name}_China.xlsx') as writer:
        X_tot.to_excel(writer, sheet_name='X-tech-construction')
        x_HVC_tot.to_excel(writer, sheet_name='x_HVC_construction')
        U_tot.to_excel(writer, sheet_name='u-tech-capacity')
        u_HVC_tot.to_excel(writer, sheet_name='u_HVC_capacity')
        y_HVC_tot.to_excel(writer, sheet_name='y_HVC_production')
        production_tot.to_excel(writer, sheet_name='production_t')
        H2_AMM_tot.to_excel(writer, sheet_name='input-KWh-AMM')
        H2_METH_tot.to_excel(writer, sheet_name='input-KWh-METH')
        H2_Oil_tot.to_excel(writer, sheet_name='input-KWh-Oil')
        HVC_FS_input_tot.to_excel(writer, sheet_name='HVC_feedstock_t')
        CO2_AMM_tot.to_excel(writer, sheet_name='CO2-emission-AMM')
        CO2_METH_tot.to_excel(writer, sheet_name='CO2-emission-METH')
        CO2_Oil_tot.to_excel(writer, sheet_name='CO2-emission-Oil')
        CO2_Oil_CC_tot.to_excel(writer, sheet_name='CO2_Oil_CC')
        CCS_amount_tot.to_excel(writer, sheet_name='CCS_amount')
        CO2_HVC_dir_tot.to_excel(writer, sheet_name='CO2_HVC_dir')
        CO2_HVC_indir_tot.to_excel(writer, sheet_name='CO2_HVC_indir')
        CE_tot.to_excel(writer, sheet_name='CO2_by_product')
        cost_by_tech_result_tot.to_excel(writer, sheet_name='CAPEX')
        cost_AMM_tot.to_excel(writer, sheet_name='cost_AMM')
        cost_METH_tot.to_excel(writer, sheet_name='cost_METH')
        cost_next_tot.to_excel(writer, sheet_name='cost_next')
        cost_Oil_tot.to_excel(writer, sheet_name='cost_Oil')
        cost_HVC_tot.to_excel(writer, sheet_name='cost_HVC')
        LCOX_tot.to_excel(writer, sheet_name='LCOX_rmb_per_t')
        LCOX_tot_with_CC.to_excel(writer, sheet_name='LCOX_wCC_rmb_per_t')
        production_value.to_excel(writer, sheet_name='production_value')
        H2_AMM_group_tot.to_excel(writer, sheet_name='H2_AMM')
        H2_METH_group_tot.to_excel(writer, sheet_name='H2_METH')
        H2_Oil_group_tot.to_excel(writer, sheet_name='H2_Oil')
        AMM_product_tot.to_excel(writer, sheet_name='AMM_product')
        METH_product_tot.to_excel(writer, sheet_name='METH_product')
        Oil_product_tot.to_excel(writer, sheet_name='Oil_product')
        trans_AMM.to_excel(writer, sheet_name='trans_AMM_2060')
        trans_METH.to_excel(writer, sheet_name='trans_METH_2060')
        trans_H2.to_excel(writer, sheet_name='trans_H2_2060')
        trans_cost_tot.to_excel(writer, sheet_name='trans_cost')
        CDR_amount_tot.to_excel(writer, sheet_name='CDR_amount')
        CDR_cost_tot.to_excel(writer, sheet_name='CDR_cost')
        heat_output_tot.to_excel(writer, sheet_name='heat_output_GJ')
        heat_input_tot.to_excel(writer, sheet_name='heat_input_GJ')
        ele_input_tot.to_excel(writer, sheet_name='ele_input_MWh')
        CO2_heat_tot.to_excel(writer, sheet_name='CO2_heat_kg')
        CO2_ele_tot.to_excel(writer, sheet_name='CO2_ele_kg')
        CO2_tot.to_excel(writer, sheet_name='CO2_sum_kg')
        cost_heat_tot.to_excel(writer, sheet_name='cost_heat')
        cost_ele_tot.to_excel(writer, sheet_name='cost_ele')
        CCS_heat_tot.to_excel(writer, sheet_name='CCS_heat')
        CCS_tot.to_excel(writer, sheet_name='CCS_tot_kg')
        CCS_cost_tot.to_excel(writer, sheet_name='CCS_cost_sum')
        SP.to_excel(writer, sheet_name='shadow prices_t')
        CDR_SP.to_excel(writer, sheet_name='CCS shadow prices')
        Bio_SP.to_excel(writer, sheet_name='Bio shadow prices_GJ')
        if GR_upper != 0:
            tech_SP.to_excel(writer, sheet_name=f'{Tech}_limit_5%AGR')

    print("Optimal solution saved to 'result.xlsx'")

In [36]:
LT=get_LT_data()
OH_max=get_OH_data()
HP=get_HP_data()
CF,CCS_delta_CF=get_CF_data()
CF_METH,CCS_delta_CF_METH=get_CF_METH_data()
EXCAP,EXCAP_process=get_EXCAP_data()
CCS_cost=get_CCS_cost_data()
trans_cost=get_trans_cost()
CO2_POT=get_CO2_POT()#no need for industrial CO2
CO2_level='M'#no need for industrial CO2
Demand_data,Supply_data,Carbon_Target,CDR_cost=get_SCE_data()
Bio_POT=get_Bio_POT()
WS_POT=get_Wind_Solar_POT()
HVC_para=get_hvc_parameters()
demand_all=create_demand_dict()
DS_ratio=get_province_demand_supply_share()
EI=get_energy_intensity()
ele_data=get_ele_data()
Heat_para=get_heat_production_para()


In [ ]:
for S in ['S3']:
    if S=='S0':#BAU
        Bio_level,Demand_level,Carbon_level,CDR_max,CDR_level,CCS_level,HN_on=['BAU','M','L',0.15,'M','CCS_limit_off','HN_off']#the potential of biomass supply,demand level, carbon emissions reduction target, CDR potential, cost level of CDR(DACCS), CCS potential, if unlock hydropower and nuclear power
    elif S=='S1':#WS-Dom
        Bio_level,Demand_level,Carbon_level,CDR_max,CDR_level,CCS_level,HN_on=['BAU','M','H',0.15,'M',600,'HN_off']
    elif S=='S3':#Div-E
        Bio_level,Demand_level,Carbon_level,CDR_max,CDR_level,CCS_level,HN_on=['MAR II','M','H',0.15,'M',600,'HN_on']
    
    POT=get_POT_data()
    CAP,OM,FS=get_cost_data('M',bio_cost='M')#the cost of ELE and biomass at basic level
    NC=get_next_cost_data('M')#the cost of DAC-based CO2 at basic level
    SCE_data={'Demand':Demand_data[Demand_level],
             'Carbon_Target':Carbon_Target[Carbon_level],
             'CDR_cost':CDR_cost[CDR_level],
             'CDR_max':CDR_max,
             'CCS':CCS_level,
             'Supply':Supply_data[Demand_level],
             'Bio_POT':Bio_POT[Bio_level],
             'HN_on':HN_on}
    store_name='{}/{}'.format(S,S)
    solve_model(CAP,OM,FS,NC,LT,OH_max,0,HP,CF,CF_METH,CCS_delta_CF,CCS_delta_CF_METH,EXCAP,EXCAP_process,POT,WS_POT,CO2_POT,CO2_level,CCS_cost,trans_cost,HVC_para,demand_all,DS_ratio,EI,ele_data,Heat_para,SCE_data,S,store_name=store_name)


The model is a Linear Programming (LP) model.
Set parameter ScaleFlag to value 1
Set parameter Method to value 2
Set parameter FeasibilityTol to value 0.001
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: Intel(R) Core(TM) i9-14900HX, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 388426 rows, 657297 columns and 3793378 nonzeros
Model fingerprint: 0x0e7d3ac8
Coefficient statistics:
  Matrix range     [4e-12, 8e+03]
  Objective range  [1e-04, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-13, 1e+12]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 344518 rows and 608977 columns
Presolve time: 0.59s
Presolved: 43908 rows, 48320 columns, 333409 nonzeros

Ordering time: 0.08s

Barrier statistics:
 AA' NZ     : 6.704e+05
 Factor NZ  : 3.794e+06 (roughly 70 MB of memor